In [1]:
import datasets
from datasets import load_dataset, load_from_disk
import scanpy as sc

/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
scrna = sc.read_h5ad("/home/share/huadjyin/home/weiyilin/project/DNALLM/datasets/embedding_datasets/scrna_dataset/scgpt/CD8_expression_5K/CD8_expression_5K_embedding.h5ad")
scrna

AnnData object with n_obs × n_vars = 359661 × 512
    obs: 'sample'

In [3]:
scrna.obs.keys()

Index(['sample'], dtype='object')

In [4]:
# ds:datasets.Dataset = load_from_disk("/home/share/huadjyin/home/weiyilin/project/DNALLM/datasets/tokenized_datasets/seq_dataset/hyenadna",
# )

In [5]:
# def gen_from_shards(shards):
#     for shard in shards:
#         for example in shard:
#             yield example

In [6]:
# shards = [ds['train'].shard(num_shards=5000, index=i) for i in range(5000)]
# iter_ds = gen_from_shards(shards)


In [8]:
ds:datasets.Dataset = load_dataset("json",
    data_files = [f"/home/share/huadjyin/home/weiyilin/project/DNALLM/datasets/tokenized_datasets/seq_dataset/hyenadna_json/validation/validation_{i}_500.json" for i in range(4)],
    trust_remote_code=True,
    split='validation',
    streaming=True,
    # num_proc=5
)

ValueError: Bad split: validation. Available splits: ['train']

In [ ]:
ds

IterableDatasetDict({
    train: IterableDataset({
        features: Unknown,
        n_shards: 4
    })
})

In [13]:

import torch
import itertools
from itertools import chain

from tqdm import tqdm

class OmicIterableDataset(torch.utils.data.IterableDataset):
    """IterableDataset that combines seq sequence, ATAC peaks and RNA. These data is undergoing process in order to feed LLM.
    """
    def __init__(
        self,
        seq_tokenized_iter_dataset: datasets.IterableDataset,
        scrna_embedding_data,
    ) -> None:
        """OmicDataset derived from torch.utils.data.Dataset 

        Parameters
        ----------
        seq_tokenized_iter_dataset : datasets.IterableDataset
            The tokenized seq iterable dataset.

            >>> Example:
            IterableDataset({
                features: ['sample', 'pos', 'peak_value', 'input_ids', 'record_id'],
                n_shards: 500
            })

        """

        super().__init__()

        self.seq_token_ds = seq_tokenized_iter_dataset
        self.scrna_embedding_data = scrna_embedding_data
    
    def _process_dataset(self, i_seq_token_ds):

        sample_id = i_seq_token_ds['sample']

        if 'sample' not in self.scrna_embedding_data.obs.keys():
            raise ValueError("The sample is not found in the scRNA embedding adata.obs.keys().")

        cell_adata = self.scrna_embedding_data[
            self.scrna_embedding_data.obs['sample'] == sample_id]
        if cell_adata.shape[0] == 0:
            raise ValueError(f"Not found scRNA embedding for sample {sample_id}")
        
        _cell_embedding = cell_adata.X.tolist()
        _cell_ids = cell_adata.obs_names.to_list()

        # return (i_seq_token_ds["input_ids"], )

        # print(i_seq_token_ds["input_ids"])

        return dict(
            seq_input_ids=i_seq_token_ds["input_ids"],
            peak_value=i_seq_token_ds["peak_value"],
            # cell_embedding =_cell_embedding, # 2d list
            # # the followings are not used in the model forward, just for recording the sample and id.
            # sample=sample_id,
            # pos=i_seq_token_ds["pos"] if "pos" in i_seq_token_ds else None,
            # cell_id = _cell_ids  # list
        )
    
    def __iter__(self):

        worker_info = torch.utils.data.get_worker_info()

        worker_id = worker_info.id if worker_info is not None else 0
        worker_total_num = worker_info.num_workers if worker_info is not None else 1

        # Map each element using the self._process_data
        mapped_itr = map(self._process_dataset, self.seq_token_ds)

        # Add multiworker functionality
        mapped_itr = itertools.islice(mapped_itr, worker_id, None, worker_total_num)

        return mapped_itr

        # return iter(map(self._process_dataset, self.seq_token_ds))

        

In [14]:
import torch.utils.data

datas = OmicIterableDataset(ds, scrna)
data_loader = torch.utils.data.DataLoader(datas, batch_size=2, num_workers=4, collate_fn=lambda x: x )

In [15]:
# next(enumerate(data_loader))

In [16]:
# from tqdm import tqdm
for i, d in enumerate(data_loader):
    if i > 3:
        break
    # print(d.keys())
    print(d)
    # print(d['peak_value'])
    # print(d['cell_embedding'])
    print("===")
    # print(d['sample'])
    # print(d.keys())
    # # print(d['cell_id'])
    # print(d['pos'])
    # print("===")
    # print(d['seq_input_ids'])
    # print(d['peak_value'])
    # print(d['cell_embedding'])

[{'seq_input_ids': [7, 10, 7, 10, 7, 10, 7, 8, 7, 8, 7, 10, 7, 10, 7, 10, 7, 10, 7, 10, 9, 10, 9, 10, 7, 10, 7, 10, 7, 10, 7, 10, 7, 10, 7, 10, 9, 10, 9, 10, 9, 10, 7, 10, 7, 10, 7, 10, 7, 10, 7, 10, 7, 10, 9, 10, 9, 10, 7, 10, 7, 10, 7, 10, 7, 10, 7, 8, 7, 8, 7, 10, 7, 10, 7, 10, 7, 10, 9, 10, 9, 10, 7, 10, 7, 10, 7, 10, 7, 10, 7, 10, 7, 10, 7, 10, 7, 8, 7, 10, 7, 10, 7, 10, 7, 10, 9, 10, 9, 10, 7, 10, 7, 10, 7, 10, 7, 10, 7, 10, 7, 8, 7, 8, 7, 10, 7, 10, 7, 10, 7, 10, 9, 10, 9, 10, 7, 10, 7, 10, 7, 10, 7, 10, 7, 10, 7, 8, 7, 10, 7, 10, 7, 10, 7, 10, 9, 10, 9, 10, 7, 10, 7, 10, 7, 10, 7, 10, 7, 10, 7, 10, 7, 8, 7, 10, 7, 10, 7, 10, 7, 10, 7, 10, 7, 10, 9, 10, 9, 10, 7, 10, 7, 10, 7, 10, 7, 8, 7, 8, 7, 10, 10, 10, 7, 7, 7, 10, 9, 9, 7, 7, 8, 8, 7, 9, 8, 7, 9, 7, 9, 10, 8, 10, 7, 9, 9, 8, 7, 9, 9, 9, 10, 10, 9, 9, 7, 7, 9, 8, 7, 8, 9, 10, 9, 10, 7, 8, 8, 10, 10, 10, 8, 8, 8, 7, 8, 10, 10, 10, 7, 10, 7, 10, 10, 10, 10, 7, 7, 7, 10, 10, 9, 10, 8, 10, 7, 7, 9, 8, 8, 7, 10, 8, 7, 7, 9, 10, 

In [18]:

from omic_config import (
    OmicLMTrainingConfig, 
    OmicRawDataConfig, 
    OmicPretrainedModelAndTokenizationConfig
)

ImportError: cannot import name 'OmicLMTrainingConfig' from 'omic_config' (/home/share/huadjyin/home/weiyilin/project/DNALLM/omiclm/omic_config/__init__.py)

In [2]:
import os
import scanpy as sc
from omic_model import (
    GeneformerDataProcessor,
    ScGPTDataProcessor
)

# scRNA data preprocess



## BY Geneformor

### Using default gene_name_id_dict.pkl file

In [3]:
raw_data_config, pretrained_model_tokenizer_config = OmicRawDataConfig(), OmicPretrainedModelAndTokenizationConfig()
print(pretrained_model_tokenizer_config.scrna_model_name)
print(pretrained_model_tokenizer_config.seq_model_name)

raw_scrna_count_file_name = raw_data_config.raw_scrna_file_name 
adata = sc.read_h5ad(raw_data_config.raw_scrna_file_name)


geneformer
hyenadna


/home/HPCBase/tools/anaconda3/lib/python3.9/site-packages/h5py/_hl/dataset.py:767: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)


In [4]:
adata


AnnData object with n_obs × n_vars = 359661 × 5000
    obs: 'library', 'sample', 'predicted_doublets', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'total_counts_ncRNA', 'pct_counts_ncRNA', 'total_counts_hb', 'pct_counts_hb', 'n_counts', 'n_genes', 'celltype_l1', 'celltype_l2', 'celltype_l3', 'celltype_l4', 'SEACell_ID', 'SEACell_l3', 'SEACell_l4', 'final_annotation'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'

In [5]:
adata.raw.var_names, adata.var_names 

(Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2ML1-AS1',
        'A2ML1-AS2', 'A3GALT2', 'A4GALT',
        ...
        'ZWINT', 'ZXDA', 'ZXDB', 'ZXDC', 'ZYG11A', 'ZYG11B', 'ZYX', 'ZZEF1',
        'hsa-mir-1253', 'hsa-mir-423'],
       dtype='object', length=36326),
 Index(['A2M-AS1', 'AAK1', 'AANAT', 'AARS2', 'AASDH', 'AASS', 'AATK', 'ABAT',
        'ABCA10', 'ABCA5',
        ...
        'ZRANB2', 'ZRANB3', 'ZRSR2', 'ZSCAN29', 'ZSCAN30', 'ZSCAN5B', 'ZSWIM6',
        'ZSWIM9', 'ZWILCH', 'ZYG11B'],
       dtype='object', length=5000))

In [6]:
# output_dir 
prep_out_dir = raw_data_config.raw_scrna_preprocessed_dir
os.makedirs(prep_out_dir, exist_ok=True)


In [7]:
# print the gene id map
import pickle
gene_id_map_file_name = os.path.join(pretrained_model_tokenizer_config.scrna_model_path, "gene_name_id_dict.pkl")
print(gene_id_map_file_name)

with open(gene_id_map_file_name, 'rb') as f:
    gene_id_map = pickle.load(f)

print(len(gene_id_map))
gene_id_map
# /home/share/huadjyin/home/weiyilin/project/DNALLM/geneformer-12L-30M/gene_name_id_dict.pkl

/home/share/huadjyin/home/weiyilin/project/DNALLM/geneformer-12L-30M/gene_name_id_dict.pkl
40248


{'MT-TF': 'ENSG00000210049',
 'MT-RNR1': 'ENSG00000211459',
 'MT-TV': 'ENSG00000210077',
 'MT-RNR2': 'ENSG00000210082',
 'MT-TL1': 'ENSG00000209082',
 'MT-ND1': 'ENSG00000198888',
 'MT-TI': 'ENSG00000210100',
 'MT-TQ': 'ENSG00000210107',
 'MT-TM': 'ENSG00000210112',
 'MT-ND2': 'ENSG00000198763',
 'MT-TW': 'ENSG00000210117',
 'MT-TA': 'ENSG00000210127',
 'MT-TN': 'ENSG00000210135',
 'MT-TC': 'ENSG00000210140',
 'MT-TY': 'ENSG00000210144',
 'MT-CO1': 'ENSG00000198804',
 'MT-TS1': 'ENSG00000210151',
 'MT-TD': 'ENSG00000210154',
 'MT-CO2': 'ENSG00000198712',
 'MT-TK': 'ENSG00000210156',
 'MT-ATP8': 'ENSG00000228253',
 'MT-ATP6': 'ENSG00000198899',
 'MT-CO3': 'ENSG00000198938',
 'MT-TG': 'ENSG00000210164',
 'MT-ND3': 'ENSG00000198840',
 'MT-TR': 'ENSG00000210174',
 'MT-ND4L': 'ENSG00000212907',
 'MT-ND4': 'ENSG00000198886',
 'MT-TH': 'ENSG00000210176',
 'MT-TS2': 'ENSG00000210184',
 'MT-TL2': 'ENSG00000210191',
 'MT-ND5': 'ENSG00000198786',
 'MT-ND6': 'ENSG00000198695',
 'MT-TE': 'ENSG00000

In [8]:
data_preprocessor = GeneformerDataProcessor(raw_adata_file_name=raw_scrna_count_file_name)
data_preprocessor.data_config

/home/HPCBase/tools/anaconda3/lib/python3.9/site-packages/h5py/_hl/dataset.py:767: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)


{'sc_data_file_name': '/home/share/huadjyin/home/weiyilin/project/DNALLM/processed_data/CD8_expression_5K.h5ad',
 'is_count_raw_data': False,
 'batch_key': None}

In [9]:
processed_adata = data_preprocessor.preprocess_data(
    gene_name_id=gene_id_map_file_name,
    has_gene_name_id_maped=False,
    use_raw=True,
    output_dir=prep_out_dir,
    min_gene_vocab_matched_frac=0.6,
    filter_cell_by_genes=10,
    filter_gene_by_cells=10,
)

gene_name not found in var columns, Using var_names instead


/home/share/huadjyin/home/weiyilin/project/DNALLM/omiclm/omic_model/pretrained_model/data_prep/_geneformer_data_processor.py:232: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  self.adata.obs['adata_order'] = self.adata.obs_names.to_list()
/home/HPCBase/tools/anaconda3/lib/python3.9/site-packages/h5py/_hl/attrs.py:103: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  self.create(name, data=value)
/home/HPCBase/tools/anaconda3/lib/python3.9/site-packages/h5py/_hl/filters.py:243: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  chunks = guess_chunk(shape, maxshape, dtype.itemsize)


In [10]:
prep_adata = sc.read_h5ad("/home/share/huadjyin/home/weiyilin/project/DNALLM/datasets/raw_datasets/scrna_processed/hyenadna/CD8_expression_5K_preprocessed.h5ad")

/home/HPCBase/tools/anaconda3/lib/python3.9/site-packages/h5py/_hl/dataset.py:767: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)


In [14]:
prep_adata.obs

,library,sample,predicted_doublets,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,total_counts_ncRNA,...,n_genes,celltype_l1,celltype_l2,celltype_l3,celltype_l4,SEACell_ID,SEACell_l3,SEACell_l4,final_annotation,adata_order
RNA-E-B21712635155-a_2_CELL17_N1,RNA-E-B21712635155-a_2,E-B21712635155,False,119,371.0,57.0,3.335284,304.0,17.788181,16.0,...,119,CD8_T,CD8_CTL,CD8_CTL,CD8_CTL_GZMH,E-B21712635155-CD8_CTL-SEACell-19,E-B21712635155-CD8_CTL-SEACell-19,E-B21712635155-CD8_CTL_GZMH-SEACell-1,CD8_CTL_GZMB,RNA-E-B21712635155-a_2_CELL17_N1
RNA-E-B21712635155-a_2_CELL76_N3,RNA-E-B21712635155-a_2,E-B21712635155,False,320,1246.0,516.0,8.963001,1072.0,18.620810,49.0,...,320,CD8_T,CD8_CTL,CD8_CTL,CD8_CTL_ITGB1,E-B21712635155-CD8_CTL-SEACell-11,E-B21712635155-CD8_CTL-SEACell-11,E-B21712635155-CD8_CTL_ITGB1-SEACell-3,CD8_CTL_GZMB,RNA-E-B21712635155-a_2_CELL76_N3
RNA-E-B21712635155-a_2_CELL114_N1,RNA-E-B21712635155-a_2,E-B21712635155,False,206,637.0,117.0,4.426788,504.0,19.069239,18.0,...,206,CD8_T,CD8_CTL,CD8_CTL,CD8_CTL_ITGB1,E-B21712635155-CD8_CTL-SEACell-20,E-B21712635155-CD8_CTL-SEACell-20,E-B21712635155-CD8_CTL_ITGB1-SEACell-4,CD8_CTL_GZMB,RNA-E-B21712635155-a_2_CELL114_N1
RNA-E-B21712635155-a_2_CELL115_N1,RNA-E-B21712635155-a_2,E-B21712635155,False,146,421.0,114.0,6.162162,252.0,13.621622,25.0,...,146,CD8_T,CD8_CTL,CD8_CTL,CD8_CTL_GZMH,E-B21712635155-CD8_CTL-SEACell-8,E-B21712635155-CD8_CTL-SEACell-8,E-B21712635155-CD8_CTL_GZMH-SEACell-11,CD8_CTL_GZMB,RNA-E-B21712635155-a_2_CELL115_N1
RNA-E-B21712635155-a_2_CELL119_N1,RNA-E-B21712635155-a_2,E-B21712635155,False,114,361.0,112.0,6.565064,355.0,20.808910,14.0,...,114,CD8_T,CD8_CTL,CD8_CTL,CD8_CTL_GZMH,E-B21712635155-CD8_CTL-SEACell-16,E-B21712635155-CD8_CTL-SEACell-16,E-B21712635155-CD8_CTL_GZMH-SEACell-0,CD8_CTL_GZMB,RNA-E-B21712635155-a_2_CELL119_N1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RNA-E-B21771966688-A_1_CELL3742_N1,RNA-E-B21771966688-A_1,E-B21771966688,False,164,378.0,59.0,2.955912,491.0,24.599199,12.0,...,164,CD8_T,CD8_CTL,CD8_CTL,CD8_CTL_GZMH,E-B21771966688-CD8_CTL-SEACell-1,E-B21771966688-CD8_CTL-SEACell-1,E-B21771966688-CD8_CTL_GZMH-SEACell-0,CD8_CTL_GZMB,RNA-E-B21771966688-A_1_CELL3742_N1
RNA-E-B21771966688-A_1_CELL4548_N1,RNA-E-B21771966688-A_1,E-B21771966688,False,91,209.0,45.0,3.943909,282.0,24.715162,4.0,...,91,CD8_T,CD8_CTL,CD8_CTL,CD8_CTL_GZMH,E-B21771966688-CD8_CTL-SEACell-1,E-B21771966688-CD8_CTL-SEACell-1,E-B21771966688-CD8_CTL_GZMH-SEACell-0,CD8_CTL_GZMB,RNA-E-B21771966688-A_1_CELL4548_N1
RNA-E-B21771966688-A_1_CELL5056_N1,RNA-E-B21771966688-A_1,E-B21771966688,False,81,195.0,62.0,5.838041,325.0,30.602636,8.0,...,81,CD8_T,CD8_CTL,CD8_CTL,CD8_CTL_GZMH,E-B21771966688-CD8_CTL-SEACell-1,E-B21771966688-CD8_CTL-SEACell-1,E-B21771966688-CD8_CTL_GZMH-SEACell-0,CD8_CTL_GZMB,RNA-E-B21771966688-A_1_CELL5056_N1
RNA-E-B21771966688-A_1_CELL5483_N1,RNA-E-B21771966688-A_1,E-B21771966688,False,207,624.0,222.0,8.872902,797.0,31.854517,21.0,...,207,CD8_T,CD8_CTL,CD8_CTL,CD8_CTL_IDO1,E-B21771966688-CD8_CTL-SEACell-1,E-B21771966688-CD8_CTL-SEACell-1,E-B21771966688-CD8_CTL_IDO1-SEACell-0,CD8_CTL_GZMB,RNA-E-B21771966688-A_1_CELL5483_N1


### Using adata with ensembl id

In [3]:
raw_data_config, pretrained_model_tokenizer_config = OmicRawDataConfig(), OmicPretrainedModelAndTokenizationConfig()
print(pretrained_model_tokenizer_config.scrna_model_name)
print(pretrained_model_tokenizer_config.seq_model_name)

raw_scrna_count_file_name = raw_data_config.raw_scrna_file_name 
adata = sc.read_h5ad(raw_data_config.raw_scrna_file_name)

geneformer
hyenadna


/home/HPCBase/tools/anaconda3/lib/python3.9/site-packages/h5py/_hl/dataset.py:767: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)


In [4]:
adata

AnnData object with n_obs × n_vars = 359661 × 5000
    obs: 'library', 'sample', 'predicted_doublets', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'total_counts_ncRNA', 'pct_counts_ncRNA', 'total_counts_hb', 'pct_counts_hb', 'n_counts', 'n_genes', 'celltype_l1', 'celltype_l2', 'celltype_l3', 'celltype_l4', 'SEACell_ID', 'SEACell_l3', 'SEACell_l4', 'final_annotation'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'ensembl_id'
    uns: 'hvg', 'log1p'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'

In [21]:
len(adata.var_names), len(adata.raw.var_names)

(5000, 36326)

In [29]:
adata.obs.total_counts

RNA-E-B21712635155-a_2_CELL17_N1      1709.0
RNA-E-B21712635155-a_2_CELL76_N3      5757.0
RNA-E-B21712635155-a_2_CELL114_N1     2643.0
RNA-E-B21712635155-a_2_CELL115_N1     1850.0
RNA-E-B21712635155-a_2_CELL119_N1     1706.0
                                       ...  
RNA-E-B21771966688-A_1_CELL3742_N1    1996.0
RNA-E-B21771966688-A_1_CELL4548_N1    1141.0
RNA-E-B21771966688-A_1_CELL5056_N1    1062.0
RNA-E-B21771966688-A_1_CELL5483_N1    2502.0
RNA-E-B21771966688-A_1_CELL5858_N1    1967.0
Name: total_counts, Length: 359661, dtype: float32

In [25]:
adata.var_names

Index(['A2M-AS1', 'AAK1', 'AANAT', 'AARS2', 'AASDH', 'AASS', 'AATK', 'ABAT',
       'ABCA10', 'ABCA5',
       ...
       'ZRANB2', 'ZRANB3', 'ZRSR2', 'ZSCAN29', 'ZSCAN30', 'ZSCAN5B', 'ZSWIM6',
       'ZSWIM9', 'ZWILCH', 'ZYG11B'],
      dtype='object', length=5000)

In [30]:
adata.X.A

array([[0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 1.0068682, 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 2.5642111, 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ]], dtype=float32)

In [31]:
adata.X = adata.raw[:, adata.var_names].X.copy()



In [32]:
adata.X.A

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 3., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [33]:
sc.pp.calculate_qc_metrics(adata, 
                                   percent_top = None, 
                                   log1p = False, 
                                   inplace = True,
                                   use_raw=False,
                                   layer=None)


In [35]:
adata.obs.total_counts, adata.shape

(RNA-E-B21712635155-a_2_CELL17_N1       371.0
 RNA-E-B21712635155-a_2_CELL76_N3      1246.0
 RNA-E-B21712635155-a_2_CELL114_N1      637.0
 RNA-E-B21712635155-a_2_CELL115_N1      421.0
 RNA-E-B21712635155-a_2_CELL119_N1      361.0
                                        ...  
 RNA-E-B21771966688-A_1_CELL3742_N1     378.0
 RNA-E-B21771966688-A_1_CELL4548_N1     209.0
 RNA-E-B21771966688-A_1_CELL5056_N1     195.0
 RNA-E-B21771966688-A_1_CELL5483_N1     624.0
 RNA-E-B21771966688-A_1_CELL5858_N1     508.0
 Name: total_counts, Length: 359661, dtype: float32,
 (359661, 5000))

In [36]:
sc.pp.filter_cells(adata, min_genes=int(10))
sc.pp.filter_genes(adata, min_cells=int(10))

In [37]:
adata.obs.total_counts, adata.shape

(RNA-E-B21712635155-a_2_CELL17_N1       371.0
 RNA-E-B21712635155-a_2_CELL76_N3      1246.0
 RNA-E-B21712635155-a_2_CELL114_N1      637.0
 RNA-E-B21712635155-a_2_CELL115_N1      421.0
 RNA-E-B21712635155-a_2_CELL119_N1      361.0
                                        ...  
 RNA-E-B21771966688-A_1_CELL3742_N1     378.0
 RNA-E-B21771966688-A_1_CELL4548_N1     209.0
 RNA-E-B21771966688-A_1_CELL5056_N1     195.0
 RNA-E-B21771966688-A_1_CELL5483_N1     624.0
 RNA-E-B21771966688-A_1_CELL5858_N1     508.0
 Name: total_counts, Length: 359661, dtype: float32,
 (359661, 4692))

In [38]:
adata.var['ensembl_id']

A2M-AS1    ENSG00000245105
AAK1       ENSG00000115977
AANAT      ENSG00000129673
AARS2      ENSG00000124608
AASDH      ENSG00000157426
                ...       
ZSCAN5B    ENSG00000197213
ZSWIM6     ENSG00000130449
ZSWIM9     ENSG00000185453
ZWILCH     ENSG00000174442
ZYG11B     ENSG00000162378
Name: ensembl_id, Length: 4692, dtype: category
Categories (3311, object): ['ENSG00000000005', 'ENSG00000000457', 'ENSG00000000971', 'ENSG00000001461', ..., 'ENSG00000285825', 'ENSG00000286920', 'ENSG00000291347', 'ENSG00000292214']

In [41]:
adata.var['has_ensembl_match'] = adata.var['ensembl_id'].notnull()

In [49]:
adata.var[adata.var['has_ensembl_match']==False]

,highly_variable,means,dispersions,dispersions_norm,ensembl_id,n_cells_by_counts,mean_counts,pct_dropout_by_counts,total_counts,n_cells,has_ensembl_match
AC000061.1,True,0.000151,1.901162,0.905456,NaN,12,0.000033,99.996664,12.0,12,False
AC000082.1,True,0.000276,2.229956,2.530286,NaN,14,0.000044,99.996107,16.0,14,False
AC000085.1,True,0.000765,1.977816,1.284264,NaN,55,0.000156,99.984708,56.0,55,False
AC002059.1,True,0.002551,2.000109,1.394431,NaN,212,0.000637,99.941056,229.0,212,False
AC002074.1,True,0.000663,2.021303,1.499166,NaN,47,0.000145,99.986932,52.0,47,False
...,...,...,...,...,...,...,...,...,...,...,...
Z98885.3,True,0.004558,1.986556,1.327453,NaN,291,0.000851,99.919090,306.0,291,False
Z99127.4,True,0.017345,1.956452,1.178689,NaN,1378,0.004421,99.616861,1590.0,1378,False
Z99916.1,True,0.000703,1.924026,1.018443,NaN,55,0.000184,99.984708,66.0,55,False
ZBED8,True,0.001872,1.989039,1.339724,NaN,153,0.000481,99.957460,173.0,153,False


In [11]:
data_preprocessor = GeneformerDataProcessor(raw_adata_file_name=raw_scrna_count_file_name)
data_preprocessor.data_config

/home/HPCBase/tools/anaconda3/lib/python3.9/site-packages/h5py/_hl/dataset.py:767: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)


{'input_data_file_name': '/home/share/huadjyin/home/weiyilin/project/DNALLM/processed_data/CD8_expression_ensembl.h5ad',
 'is_count_raw_data': False,
 'batch_key': None}

In [12]:
# output_dir 
prep_out_dir = raw_data_config.raw_scrna_preprocessed_dir
os.makedirs(prep_out_dir, exist_ok=True)

processed_adata = data_preprocessor.preprocess_data(
    gene_name_id=None,
    has_gene_name_id_maped=True,
    use_raw=True,
    output_dir=prep_out_dir,
    min_gene_vocab_matched_frac=0.6,
    filter_cell_by_genes=10,
    filter_gene_by_cells=10,
)


gene_name not found in var columns, Using var_names instead
You provided adata with ensembl id, ignoring gene name mapping.


/home/share/huadjyin/home/weiyilin/project/DNALLM/omiclm/omic_model/pretrained_model/data_prep/_geneformer_data_processor.py:233: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  self.adata.obs['adata_order'] = self.adata.obs_names.to_list()
Matched 70.57% genes (3311/4692) genes after cell and gene filtering. Note, you provided adata with ensembl id, which might be potential issue as the provided ensembl ids would be different from ensembl ids in the vocab provided by Genformer.
/home/HPCBase/tools/anaconda3/lib/python3.9/site-packages/h5py/_hl/attrs.py:103: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  self.create(name, data=value)
/home/HPCBase/tools/anaconda3/lib/python3.9/site-packages/h5py/_hl/filters.py:243: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  chunks = gues

In [14]:
processed_adata = sc.read_h5ad("/home/share/huadjyin/home/weiyilin/project/DNALLM/datasets/raw_datasets/scrna_processed/hyenadna/CD8_expression_ensembl_preprocessed.h5ad")

/home/HPCBase/tools/anaconda3/lib/python3.9/site-packages/h5py/_hl/dataset.py:767: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)


In [16]:
processed_adata.var_names

Index(['A2M-AS1', 'AAK1', 'AANAT', 'AARS2', 'AASDH', 'AASS', 'AATK', 'ABAT',
       'ABCA10', 'ABCA5',
       ...
       'ZRANB2', 'ZRANB3', 'ZRSR2', 'ZSCAN29', 'ZSCAN30', 'ZSCAN5B', 'ZSWIM6',
       'ZSWIM9', 'ZWILCH', 'ZYG11B'],
      dtype='object', length=3311)

## BY scGPT

In [3]:
raw_data_config, pretrained_model_tokenizer_config = OmicRawDataConfig(), OmicPretrainedModelAndTokenizationConfig()
print(pretrained_model_tokenizer_config.scrna_model_name)
print(pretrained_model_tokenizer_config.seq_model_name)

raw_scrna_count_file_name = raw_data_config.raw_scrna_file_name 
adata = sc.read_h5ad(raw_data_config.raw_scrna_file_name)

sc_model_dir = pretrained_model_tokenizer_config.scrna_model_path

scgpt
hyenadna


/home/HPCBase/tools/anaconda3/lib/python3.9/site-packages/h5py/_hl/dataset.py:767: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)


In [4]:
data_preprocessor = ScGPTDataProcessor(raw_adata_file_name=raw_scrna_count_file_name)
data_preprocessor.data_config

/home/HPCBase/tools/anaconda3/lib/python3.9/site-packages/h5py/_hl/dataset.py:767: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)


{'input_data_file_name': '/home/share/huadjyin/home/weiyilin/project/DNALLM/processed_data/CD8_expression_ensembl.h5ad',
 'is_count_raw_data': False,
 'batch_key': None}

In [5]:
gene_vocab_fname = os.path.join(sc_model_dir, "vocab.json")
prep_out_dir = raw_data_config.raw_scrna_preprocessed_dir

processed_adata = data_preprocessor.preprocess_data(
    gene_vocab=gene_vocab_fname,
    output_dir=prep_out_dir,
    use_raw=True,
    min_gene_vocab_matched_frac=0.6,
    filter_gene_by_counts=3,
    filter_cell_by_counts=False,
    normalize_total=1e4,
    n_hvg=0,
    n_bins=51,
    normed_key="X_normed",
    log1p_key="X_log1p",
    binned_key="X_binned",
)


gene_name not found in var columns, Using var_names instead


scGPT - INFO - Filtering genes by counts ...
scGPT - INFO - Normalizing total counts ...
scGPT - INFO - Log1p transforming ...
scGPT - INFO - Binning data ...


/home/HPCBase/tools/anaconda3/lib/python3.9/site-packages/h5py/_hl/attrs.py:103: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  self.create(name, data=value)
/home/HPCBase/tools/anaconda3/lib/python3.9/site-packages/h5py/_hl/filters.py:243: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  chunks = guess_chunk(shape, maxshape, dtype.itemsize)


In [6]:
processed_adata

AnnData object with n_obs × n_vars = 359661 × 3495
    obs: 'library', 'sample', 'predicted_doublets', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'total_counts_ncRNA', 'pct_counts_ncRNA', 'total_counts_hb', 'pct_counts_hb', 'n_counts', 'n_genes', 'celltype_l1', 'celltype_l2', 'celltype_l3', 'celltype_l4', 'SEACell_ID', 'SEACell_l3', 'SEACell_l4', 'final_annotation'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'ensembl_id', 'gene_name', 'id_in_vocab', 'n_counts'
    uns: 'hvg', 'log1p'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap', 'bin_edges'
    layers: 'X_normed', 'X_log1p', 'X_binned'

# Tokenization

## BY Geneformer

In [9]:
import pickle
v_file = "/home/share/huadjyin/home/weiyilin/project/DNALLM/geneformer-12L-30M/token_dictionary.pkl"
with open(v_file, "rb") as f:
    _vocab = pickle.load(f)
_vocab

{'<pad>': 0,
 '<mask>': 1,
 'ENSG00000000003': 2,
 'ENSG00000000005': 3,
 'ENSG00000000419': 4,
 'ENSG00000000457': 5,
 'ENSG00000000460': 6,
 'ENSG00000000938': 7,
 'ENSG00000000971': 8,
 'ENSG00000001036': 9,
 'ENSG00000001084': 10,
 'ENSG00000001167': 11,
 'ENSG00000001460': 12,
 'ENSG00000001461': 13,
 'ENSG00000001497': 14,
 'ENSG00000001561': 15,
 'ENSG00000001617': 16,
 'ENSG00000001626': 17,
 'ENSG00000001629': 18,
 'ENSG00000001630': 19,
 'ENSG00000001631': 20,
 'ENSG00000002016': 21,
 'ENSG00000002330': 22,
 'ENSG00000002549': 23,
 'ENSG00000002586': 24,
 'ENSG00000002587': 25,
 'ENSG00000002726': 26,
 'ENSG00000002745': 27,
 'ENSG00000002746': 28,
 'ENSG00000002822': 29,
 'ENSG00000002834': 30,
 'ENSG00000002919': 31,
 'ENSG00000002933': 32,
 'ENSG00000003056': 33,
 'ENSG00000003096': 34,
 'ENSG00000003137': 35,
 'ENSG00000003147': 36,
 'ENSG00000003249': 37,
 'ENSG00000003393': 38,
 'ENSG00000003400': 39,
 'ENSG00000003402': 40,
 'ENSG00000003436': 41,
 'ENSG00000003509': 4

In [11]:
len(_vocab), _vocab.get('<pad>')

(25426, 0)

sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x4003b7c30b80>


In [3]:
raw_data_config, pretrained_model_tokenizer_config = OmicRawDataConfig(), OmicPretrainedModelAndTokenizationConfig()
print(pretrained_model_tokenizer_config.scrna_model_name)
print(pretrained_model_tokenizer_config.seq_model_name)

raw_scrna_count_file_name = raw_data_config.raw_scrna_file_name 
adata = sc.read_h5ad(raw_data_config.raw_scrna_file_name)

sc_model_dir = pretrained_model_tokenizer_config.scrna_model_path

prep_out_dir = raw_data_config.raw_scrna_preprocessed_dir
adata_file_base_name = os.path.basename(raw_scrna_count_file_name).split('.')[0]
precessed_adata_file_name = os.path.join(prep_out_dir, f"{adata_file_base_name}_preprocessed.h5ad")

preprocessed_adata = sc.read_h5ad(precessed_adata_file_name)

geneformer
hyenadna


/home/HPCBase/tools/anaconda3/lib/python3.9/site-packages/h5py/_hl/dataset.py:767: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)
/home/HPCBase/tools/anaconda3/lib/python3.9/site-packages/h5py/_hl/dataset.py:767: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)


In [4]:
preprocessed_adata

AnnData object with n_obs × n_vars = 359661 × 3311
    obs: 'library', 'sample', 'predicted_doublets', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'total_counts_ncRNA', 'pct_counts_ncRNA', 'total_counts_hb', 'pct_counts_hb', 'n_counts', 'n_genes', 'celltype_l1', 'celltype_l2', 'celltype_l3', 'celltype_l4', 'SEACell_ID', 'SEACell_l3', 'SEACell_l4', 'final_annotation', 'adata_order'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'ensembl_id', 'gene_name', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_cells', 'has_ensembl_match'
    uns: 'hvg', 'log1p'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'

In [5]:
preprocessed_adata.obs

,library,sample,predicted_doublets,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,total_counts_ncRNA,...,n_genes,celltype_l1,celltype_l2,celltype_l3,celltype_l4,SEACell_ID,SEACell_l3,SEACell_l4,final_annotation,adata_order
RNA-E-B21712635155-a_2_CELL17_N1,RNA-E-B21712635155-a_2,E-B21712635155,False,119,371.0,57.0,3.335284,304.0,17.788181,16.0,...,119,CD8_T,CD8_CTL,CD8_CTL,CD8_CTL_GZMH,E-B21712635155-CD8_CTL-SEACell-19,E-B21712635155-CD8_CTL-SEACell-19,E-B21712635155-CD8_CTL_GZMH-SEACell-1,CD8_CTL_GZMB,RNA-E-B21712635155-a_2_CELL17_N1
RNA-E-B21712635155-a_2_CELL76_N3,RNA-E-B21712635155-a_2,E-B21712635155,False,320,1246.0,516.0,8.963001,1072.0,18.620810,49.0,...,320,CD8_T,CD8_CTL,CD8_CTL,CD8_CTL_ITGB1,E-B21712635155-CD8_CTL-SEACell-11,E-B21712635155-CD8_CTL-SEACell-11,E-B21712635155-CD8_CTL_ITGB1-SEACell-3,CD8_CTL_GZMB,RNA-E-B21712635155-a_2_CELL76_N3
RNA-E-B21712635155-a_2_CELL114_N1,RNA-E-B21712635155-a_2,E-B21712635155,False,206,637.0,117.0,4.426788,504.0,19.069239,18.0,...,206,CD8_T,CD8_CTL,CD8_CTL,CD8_CTL_ITGB1,E-B21712635155-CD8_CTL-SEACell-20,E-B21712635155-CD8_CTL-SEACell-20,E-B21712635155-CD8_CTL_ITGB1-SEACell-4,CD8_CTL_GZMB,RNA-E-B21712635155-a_2_CELL114_N1
RNA-E-B21712635155-a_2_CELL115_N1,RNA-E-B21712635155-a_2,E-B21712635155,False,146,421.0,114.0,6.162162,252.0,13.621622,25.0,...,146,CD8_T,CD8_CTL,CD8_CTL,CD8_CTL_GZMH,E-B21712635155-CD8_CTL-SEACell-8,E-B21712635155-CD8_CTL-SEACell-8,E-B21712635155-CD8_CTL_GZMH-SEACell-11,CD8_CTL_GZMB,RNA-E-B21712635155-a_2_CELL115_N1
RNA-E-B21712635155-a_2_CELL119_N1,RNA-E-B21712635155-a_2,E-B21712635155,False,114,361.0,112.0,6.565064,355.0,20.808910,14.0,...,114,CD8_T,CD8_CTL,CD8_CTL,CD8_CTL_GZMH,E-B21712635155-CD8_CTL-SEACell-16,E-B21712635155-CD8_CTL-SEACell-16,E-B21712635155-CD8_CTL_GZMH-SEACell-0,CD8_CTL_GZMB,RNA-E-B21712635155-a_2_CELL119_N1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RNA-E-B21771966688-A_1_CELL3742_N1,RNA-E-B21771966688-A_1,E-B21771966688,False,164,378.0,59.0,2.955912,491.0,24.599199,12.0,...,164,CD8_T,CD8_CTL,CD8_CTL,CD8_CTL_GZMH,E-B21771966688-CD8_CTL-SEACell-1,E-B21771966688-CD8_CTL-SEACell-1,E-B21771966688-CD8_CTL_GZMH-SEACell-0,CD8_CTL_GZMB,RNA-E-B21771966688-A_1_CELL3742_N1
RNA-E-B21771966688-A_1_CELL4548_N1,RNA-E-B21771966688-A_1,E-B21771966688,False,91,209.0,45.0,3.943909,282.0,24.715162,4.0,...,91,CD8_T,CD8_CTL,CD8_CTL,CD8_CTL_GZMH,E-B21771966688-CD8_CTL-SEACell-1,E-B21771966688-CD8_CTL-SEACell-1,E-B21771966688-CD8_CTL_GZMH-SEACell-0,CD8_CTL_GZMB,RNA-E-B21771966688-A_1_CELL4548_N1
RNA-E-B21771966688-A_1_CELL5056_N1,RNA-E-B21771966688-A_1,E-B21771966688,False,81,195.0,62.0,5.838041,325.0,30.602636,8.0,...,81,CD8_T,CD8_CTL,CD8_CTL,CD8_CTL_GZMH,E-B21771966688-CD8_CTL-SEACell-1,E-B21771966688-CD8_CTL-SEACell-1,E-B21771966688-CD8_CTL_GZMH-SEACell-0,CD8_CTL_GZMB,RNA-E-B21771966688-A_1_CELL5056_N1
RNA-E-B21771966688-A_1_CELL5483_N1,RNA-E-B21771966688-A_1,E-B21771966688,False,207,624.0,222.0,8.872902,797.0,31.854517,21.0,...,207,CD8_T,CD8_CTL,CD8_CTL,CD8_CTL_IDO1,E-B21771966688-CD8_CTL-SEACell-1,E-B21771966688-CD8_CTL-SEACell-1,E-B21771966688-CD8_CTL_IDO1-SEACell-0,CD8_CTL_GZMB,RNA-E-B21771966688-A_1_CELL5483_N1


In [6]:
precessed_adata_file_name, sc_model_dir, prep_out_dir,pretrained_model_tokenizer_config.tokenized_scrna_dataset_dir

('/home/share/huadjyin/home/weiyilin/project/DNALLM/datasets/raw_datasets/scrna_processed/geneformer/CD8_expression_ensembl_preprocessed.h5ad',
 '/home/share/huadjyin/home/weiyilin/project/DNALLM/geneformer-12L-30M',
 '/home/share/huadjyin/home/weiyilin/project/DNALLM/datasets/raw_datasets/scrna_processed/geneformer',
 '/home/share/huadjyin/home/weiyilin/project/DNALLM/datasets/tokenized_datasets/scrna_dataset/geneformer')

In [7]:
tokenizer_cls = pretrained_model_tokenizer_config.scrna_toknenizer_cls

In [8]:
tokenizer_cls

omic_model.pretrained_model.geneformer._geneformer_tokenizer.GeneformerTokenizerWrapper

In [9]:
tokenizer = tokenizer_cls.from_pretrained(
    sc_model_dir, 
    local_files_only=True)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'GeneformerTokenizerWrapper'.


In [10]:
tokenizer.mask_token_id

1

In [11]:
# Geneformer parameters
tokenized_dataset = tokenizer(        
    adata_full_file_name=precessed_adata_file_name,
    save_dataset_dir=pretrained_model_tokenizer_config.tokenized_scrna_dataset_dir,
    cell_type_col="celltype_l4",
    columns_to_keep=["sample", "adata_order"],
    num_workers = 4,
)

using h5ad file. This sometimes causes issues. If not working try with loom.
/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/HPCBase/tools/anaconda3/lib/python3.9/site-packages/h5py/_hl/dataset.py:767: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)


Tokenizing /home/share/huadjyin/home/weiyilin/project/DNALLM/datasets/raw_datasets/scrna_processed/geneformer/CD8_expression_ensembl_preprocessed.h5ad


/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/geneformer/tokenizer.py:230: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  for i in adata.var["ensembl_id"][coding_miRNA_loc]
/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/geneformer/tokenizer.py:233: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coding_miRNA_ids = adata.var["ensembl_id"][coding_miRNA_loc]


/home/share/huadjyin/home/weiyilin/project/DNALLM/datasets/raw_datasets/scrna_processed/geneformer/CD8_expression_ensembl_preprocessed.h5ad has no column attribute 'filter_pass'; tokenizing all cells.
Creating dataset.


Map (num_proc=4):   0%|          | 0/359661 [00:00<?, ? examples/s]

Traceback (most recent call last):
  File "/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/multiprocess/managers.py", line 591, in _run_server
    server.serve_forever()
  File "/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/multiprocess/managers.py", line 184, in serve_forever
    sys.exit(0)
SystemExit: 0

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/multiprocess/util.py", line 300, in _run_finalizers
    finalizer()
  File "/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/m

Saving the dataset (0/1 shards):   0%|          | 0/359661 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

In [ ]:
tokenized_dataset[0]

## BY scGPT

In [5]:
raw_data_config, pretrained_model_tokenizer_config = OmicRawDataConfig(), OmicPretrainedModelAndTokenizationConfig()
print(pretrained_model_tokenizer_config.scrna_model_name)
print(pretrained_model_tokenizer_config.seq_model_name)

raw_scrna_count_file_name = raw_data_config.raw_scrna_file_name 
adata = sc.read_h5ad(raw_data_config.raw_scrna_file_name)

sc_model_dir = pretrained_model_tokenizer_config.scrna_model_path

prep_out_dir = raw_data_config.raw_scrna_preprocessed_dir
adata_file_base_name = os.path.basename(raw_scrna_count_file_name).split('.')[0]
precessed_adata_file_name = os.path.join(prep_out_dir, f"{adata_file_base_name}_preprocessed.h5ad")

preprocessed_adata = sc.read_h5ad(precessed_adata_file_name)

scgpt
hyenadna


/home/HPCBase/tools/anaconda3/lib/python3.9/site-packages/h5py/_hl/dataset.py:767: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)
/home/HPCBase/tools/anaconda3/lib/python3.9/site-packages/h5py/_hl/dataset.py:767: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)


In [6]:
precessed_adata_file_name, sc_model_dir, prep_out_dir,pretrained_model_tokenizer_config.tokenized_scrna_dataset_dir

('/home/share/huadjyin/home/weiyilin/project/DNALLM/datasets/raw_datasets/scrna_processed/scgpt/CD8_expression_ensembl_preprocessed.h5ad',
 '/home/share/huadjyin/home/weiyilin/project/DNALLM/scGPT/scGPT_human',
 '/home/share/huadjyin/home/weiyilin/project/DNALLM/datasets/raw_datasets/scrna_processed/scgpt',
 '/home/share/huadjyin/home/weiyilin/project/DNALLM/datasets/tokenized_datasets/scrna_dataset/scgpt')

In [7]:
preprocessed_adata

AnnData object with n_obs × n_vars = 359661 × 3495
    obs: 'library', 'sample', 'predicted_doublets', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'total_counts_ncRNA', 'pct_counts_ncRNA', 'total_counts_hb', 'pct_counts_hb', 'n_counts', 'n_genes', 'celltype_l1', 'celltype_l2', 'celltype_l3', 'celltype_l4', 'SEACell_ID', 'SEACell_l3', 'SEACell_l4', 'final_annotation'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'ensembl_id', 'gene_name', 'id_in_vocab', 'n_counts'
    uns: 'hvg', 'log1p'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap', 'bin_edges'
    layers: 'X_binned', 'X_log1p', 'X_normed'

In [8]:
tokenizer_cls = pretrained_model_tokenizer_config.scrna_toknenizer_cls
tokenizer_cls

omic_model.pretrained_model.scgpt._scgpt_tokenizer.ScGPTTokenizerWrapper

In [9]:
from scgpt.tokenizer.gene_tokenizer import GeneVocab

_vocab2id = GeneVocab.from_file("/home/share/huadjyin/home/weiyilin/project/DNALLM/scGPT/scGPT_human/vocab.json")

In [10]:
genes = preprocessed_adata.var['gene_name'].tolist()

In [11]:
import numpy as np
gene_ids = np.array(_vocab2id(genes), dtype=int)

In [12]:
gene_ids

array([    4,    21,    24, ..., 36553, 36556, 36562])

In [13]:
tokenizer = tokenizer_cls.from_pretrained(
    sc_model_dir, 
    local_files_only=True,
    model_max_length=5000)

In [14]:
tokenizer.pad_value

-2

In [23]:
# Geneformer parameters
tokenized_dataset = tokenizer(        
    adata=preprocessed_adata, 
    input_layer_key="X_binned",
    include_zero_genes= False,
    gene_col="gene_name",
    padding=True,
    return_pt=True, 
)

In [26]:
x = [60695,    88,    91,  1319,  1393,  1441,  1784,  2017,  2383,  2895,
         2945,  3037,  4093,  4248,  4678,  4888,  4985,  5030,  5563,  7138,
         7496,  7501,  7747,  7833,  7899,  7935,  8097,  8241,  8358,  8482,
         8546,  8994,  9270,  9285,  9314,  9586,  9705,  9893,  9918, 10019,
        10196, 10245, 10583, 10676, 10784, 10810, 10833, 11293, 11323, 11417,
        11958, 11966, 12082, 12211, 12243, 12284, 12333, 12343, 12391, 12713,
        12733, 12828, 13425, 15734, 16099, 16151, 16170, 16389, 16395, 16437,
        16857, 17067, 17072, 17073, 17074, 17075, 17076, 17078, 17338, 17363,
        17783, 18166, 20376, 20381, 20433, 20860, 21064, 21348, 21559, 30485,
        30595, 30859, 31758, 31760, 31906, 32021, 32233, 32297, 32366, 32431,
        32721, 32741, 32912, 33661, 33798, 34152, 34303, 34714, 34880, 35118,
        35711, 35739, 36211, 36323, 36428, 36488]
len(x)

116

In [25]:
tokenized_dataset['genes'][0]

tensor([60695,    88,    91,  1319,  1393,  1441,  1784,  2017,  2383,  2895,
         2945,  3037,  4093,  4248,  4678,  4888,  4985,  5030,  5563,  7138,
         7496,  7501,  7747,  7833,  7899,  7935,  8097,  8241,  8358,  8482,
         8546,  8994,  9270,  9285,  9314,  9586,  9705,  9893,  9918, 10019,
        10196, 10245, 10583, 10676, 10784, 10810, 10833, 11293, 11323, 11417,
        11958, 11966, 12082, 12211, 12243, 12284, 12333, 12343, 12391, 12713,
        12733, 12828, 13425, 15734, 16099, 16151, 16170, 16389, 16395, 16437,
        16857, 17067, 17072, 17073, 17074, 17075, 17076, 17078, 17338, 17363,
        17783, 18166, 20376, 20381, 20433, 20860, 21064, 21348, 21559, 30485,
        30595, 30859, 31758, 31760, 31906, 32021, 32233, 32297, 32366, 32431,
        32721, 32741, 32912, 33661, 33798, 34152, 34303, 34714, 34880, 35118,
        35711, 35739, 36211, 36323, 36428, 36488, 60694, 60694, 60694, 60694,
        60694, 60694, 60694, 60694, 60694, 60694, 60694, 60694, 

In [22]:
tokenized_dataset[0]

(tensor([60695,    88,    91,  1319,  1393,  1441,  1784,  2017,  2383,  2895,
          2945,  3037,  4093,  4248,  4678,  4888,  4985,  5030,  5563,  7138,
          7496,  7501,  7747,  7833,  7899,  7935,  8097,  8241,  8358,  8482,
          8546,  8994,  9270,  9285,  9314,  9586,  9705,  9893,  9918, 10019,
         10196, 10245, 10583, 10676, 10784, 10810, 10833, 11293, 11323, 11417,
         11958, 11966, 12082, 12211, 12243, 12284, 12333, 12343, 12391, 12713,
         12733, 12828, 13425, 15734, 16099, 16151, 16170, 16389, 16395, 16437,
         16857, 17067, 17072, 17073, 17074, 17075, 17076, 17078, 17338, 17363,
         17783, 18166, 20376, 20381, 20433, 20860, 21064, 21348, 21559, 30485,
         30595, 30859, 31758, 31760, 31906, 32021, 32233, 32297, 32366, 32431,
         32721, 32741, 32912, 33661, 33798, 34152, 34303, 34714, 34880, 35118,
         35711, 35739, 36211, 36323, 36428, 36488]),
 tensor([ 0., 44.,  6., 11., 41.,  9., 28., 28., 12., 16., 49., 32.,  1., 27.,

In [20]:
tokenized_dataset[0][1].shape

(116,)

## BY HyenaDNA

In [7]:
from datasets import load_from_disk

raw_data_config, pretrained_model_tokenizer_config = OmicRawDataConfig(), OmicPretrainedModelAndTokenizationConfig()
print(pretrained_model_tokenizer_config.scrna_model_name)
print(pretrained_model_tokenizer_config.seq_model_name)

raw_seq_dataset_path = raw_data_config.raw_seq_dataset_dir

seq_model_dir = pretrained_model_tokenizer_config.seq_model_path
seq_tokenization_dir = pretrained_model_tokenizer_config.tokenized_seq_dataset_dir

geneformer
hyenadna


In [8]:
raw_seq_dataset = load_from_disk(raw_seq_dataset_path)
raw_seq_dataset

DatasetDict({
    train: Dataset({
        features: ['sample', 'pos', 'seq', 'peak_value'],
        num_rows: 99999
    })
    test: Dataset({
        features: ['sample', 'pos', 'seq', 'peak_value'],
        num_rows: 49999
    })
})

In [9]:
seq_model_dir, seq_tokenization_dir

('/home/share/huadjyin/home/weiyilin/project/DNALLM/HyenaDNA/hyenadna-small-32k-seqlen',
 '/home/share/huadjyin/home/weiyilin/project/DNALLM/datasets/tokenized_datasets/seq_dataset/hyenadna')

In [10]:
tokenizer_cls = pretrained_model_tokenizer_config.seq_tokenizer_cls
tokenizer_cls


omic_model.pretrained_model.hyenadna._hyenadna_tokenizer.HyenaDNACharacterTokenizer

In [11]:
# tokenizer = tokenizer_cls.from_pretrained(
#     seq_model_dir, 
#     local_files_only=True,
#     model_max_length=pretrained_model_tokenizer_config.seq_model_input_max_len)

tokenizer = tokenizer_cls(
    characters=['A', 'C', 'G', 'T', 'N'],  # add DNA characters
    model_max_length=pretrained_model_tokenizer_config.seq_model_input_max_len
)

In [12]:
def tokenize_fn(untokenized_dataset):
    token_enodes = tokenizer(
        untokenized_dataset['seq'],
        max_length=pretrained_model_tokenizer_config.seq_model_input_max_len,
        padding=False,
        truncation=False,
        add_special_tokens=False, # default value in the __call__ function
    )

    return {
        'input_ids': token_enodes['input_ids'],
        'attention_mask': token_enodes['attention_mask'],
    }

In [13]:
load_from_cache_file = True
dataset_col_remove = ['seq']
tokenized_dataset = raw_seq_dataset.map(
    tokenize_fn,
    batched=True, 
    remove_columns=dataset_col_remove,
    load_from_cache_file=load_from_cache_file, 
    num_proc=4,
    desc="Running tokenizer on raw dataset (map), no streaming",
)
tokenized_dataset.cleanup_cache_files()


Running tokenizer on raw dataset (map), no streaming (num_proc=4):   0%|          | 0/99999 [00:00<?, ? exampl…

Traceback (most recent call last):
  File "/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/multiprocess/managers.py", line 591, in _run_server
    server.serve_forever()
  File "/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/multiprocess/managers.py", line 184, in serve_forever
    sys.exit(0)
SystemExit: 0

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/multiprocess/util.py", line 300, in _run_finalizers
    finalizer()
  File "/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/m

Running tokenizer on raw dataset (map), no streaming (num_proc=4):   0%|          | 0/49999 [00:00<?, ? exampl…

Traceback (most recent call last):
  File "/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/multiprocess/managers.py", line 591, in _run_server
    server.serve_forever()
  File "/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/multiprocess/managers.py", line 184, in serve_forever
    sys.exit(0)
SystemExit: 0

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/multiprocess/util.py", line 300, in _run_finalizers
    finalizer()
  File "/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/m

{'train': 4, 'test': 4}

In [14]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['sample', 'pos', 'peak_value', 'input_ids', 'attention_mask'],
        num_rows: 99999
    })
    test: Dataset({
        features: ['sample', 'pos', 'peak_value', 'input_ids', 'attention_mask'],
        num_rows: 49999
    })
})

In [15]:
tokenized_dataset.save_to_disk(seq_tokenization_dir)
print(tokenized_dataset['train'][0])

Saving the dataset (0/1 shards):   0%|          | 0/99999 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/49999 [00:00<?, ? examples/s]

{'sample': 'E-B21846437559', 'pos': 'chr1:9928-10429', 'peak_value': 0.0037593984962406, 'input_ids': [0, 7, 8, 8, 9, 9, 9, 8, 7, 9, 8, 9, 9, 8, 8, 9, 8, 9, 9, 7, 7, 8, 7, 8, 8, 7, 9, 8, 10, 10, 9, 9, 8, 9, 8, 7, 9, 9, 8, 10, 10, 8, 10, 8, 9, 9, 10, 8, 7, 9, 9, 7, 7, 8, 9, 9, 10, 8, 8, 8, 9, 9, 9, 8, 8, 10, 8, 8, 8, 9, 8, 8, 8, 9, 8, 8, 10, 8, 8, 8, 10, 8, 8, 7, 9, 8, 8, 8, 8, 10, 8, 8, 9, 9, 9, 10, 8, 8, 8, 8, 10, 7, 8, 10, 10, 8, 9, 8, 8, 8, 8, 9, 8, 8, 7, 9, 9, 8, 8, 8, 8, 8, 7, 8, 9, 7, 8, 8, 8, 10, 7, 8, 10, 10, 8, 8, 8, 9, 8, 9, 9, 8, 8, 8, 8, 9, 9, 7, 8, 9, 8, 8, 10, 8, 8, 10, 8, 7, 8, 8, 10, 9, 8, 9, 7, 9, 8, 8, 9, 8, 8, 8, 10, 8, 8, 8, 9, 9, 7, 7, 9, 8, 10, 8, 8, 8, 9, 8, 8, 9, 8, 8, 9, 8, 10, 10, 8, 8, 9, 8, 10, 8, 10, 9, 8, 8, 9, 9, 7, 9, 8, 8, 9, 8, 10, 9, 9, 9, 10, 8, 8, 10, 7, 9, 8, 8, 8, 8, 9, 8, 8, 9, 8, 8, 8, 8, 8, 7, 9, 10, 8, 8, 9, 8, 8, 8, 9, 8, 9, 8, 8, 10, 8, 8, 9, 9, 9, 10, 8, 8, 10, 7, 7, 8, 9, 8, 8, 9, 8, 8, 9, 8, 10, 8, 9, 8, 8, 8, 10, 8, 8, 7, 8, 10, 9, 8, 9,

# Embedding

## BY Geneformer

In [3]:
import scanpy as sc

raw_data_config, pretrained_model_tokenizer_config = OmicRawDataConfig(), OmicPretrainedModelAndTokenizationConfig()
print(pretrained_model_tokenizer_config.scrna_model_name)
print(pretrained_model_tokenizer_config.seq_model_name)

raw_scrna_count_file_name = raw_data_config.raw_scrna_file_name 

sc_model_dir = pretrained_model_tokenizer_config.scrna_model_path

sc_tokenized_dataset_dir = pretrained_model_tokenizer_config.tokenized_scrna_dataset_dir
adata_file_base_name = os.path.basename(raw_scrna_count_file_name).split('.')[0]
sc_tokenized_dataset_file_name = os.path.join(sc_tokenized_dataset_dir, f"{adata_file_base_name}_preprocessed.dataset")

adata_file_base_name = os.path.basename(raw_scrna_count_file_name).split('.')[0]
precessed_adata_file_name = os.path.join(raw_data_config.raw_scrna_preprocessed_dir, f"{adata_file_base_name}_preprocessed.h5ad")

preprocessed_adata = sc.read_h5ad(precessed_adata_file_name)


geneformer
hyenadna


/home/HPCBase/tools/anaconda3/lib/python3.9/site-packages/h5py/_hl/dataset.py:767: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)


In [4]:
sc_tokenized_dataset_file_name, precessed_adata_file_name

('/home/share/huadjyin/home/weiyilin/project/DNALLM/datasets/tokenized_datasets/scrna_dataset/geneformer/CD8_expression_ensembl_preprocessed.dataset',
 '/home/share/huadjyin/home/weiyilin/project/DNALLM/datasets/raw_datasets/scrna_processed/geneformer/CD8_expression_ensembl_preprocessed.h5ad')

In [5]:
sc_model_cls =  pretrained_model_tokenizer_config.scrna_model_cls
sc_model_cls

omic_model.pretrained_model.geneformer._geneformer_model.GeneformerModelWrapper

In [6]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [7]:
sc_model = sc_model_cls.from_pretrained(
    pretrained_model_name_path=sc_model_dir,
    output_attentions=False,
    output_hidden_states=True,
    local_files_only=True,
    device=device,
)
sc_model.model

/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(25426, 512, padding_idx=0)
      (position_embeddings): Embedding(2048, 512)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.02, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.02, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-12, elementwis

In [8]:
sc_model.config

BertConfig {
  "_name_or_path": "/home/share/huadjyin/home/weiyilin/project/DNALLM/geneformer-12L-30M",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.02,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "relu",
  "hidden_dropout_prob": 0.02,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 2048,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.40.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 25426
}

In [9]:
next(sc_model.parameters()).device

device(type='cuda', index=0)

In [10]:
from datasets import load_from_disk

tokenized_dataset = load_from_disk(sc_tokenized_dataset_file_name)
embeded_adata = sc_model.extract_sample_embedding(
    tokenized_dataset=tokenized_dataset,
    adata=preprocessed_adata,
    batch_size = 48,
    embedding_key = "cell_embedding",
    layer=-2,
    pad_token_id=0,
)

Geneformer (extracting embeddings):   0%|          | 0/7493 [00:00<?, ?it/s]

AttributeError: 'GeneformerModelWrapper' object has no attribute 'tokenized_dataset'

sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x4002cfb54a60>


## BY scGPT

In [3]:
raw_data_config, pretrained_model_tokenizer_config = OmicRawDataConfig(), OmicPretrainedModelAndTokenizationConfig()
print(pretrained_model_tokenizer_config.scrna_model_name)
print(pretrained_model_tokenizer_config.seq_model_name)

raw_scrna_count_file_name = raw_data_config.raw_scrna_file_name 
adata = sc.read_h5ad(raw_data_config.raw_scrna_file_name)

sc_model_dir = pretrained_model_tokenizer_config.scrna_model_path

prep_out_dir = raw_data_config.raw_scrna_preprocessed_dir
adata_file_base_name = os.path.basename(raw_scrna_count_file_name).split('.')[0]
precessed_adata_file_name = os.path.join(prep_out_dir, f"{adata_file_base_name}_preprocessed.h5ad")

preprocessed_adata = sc.read_h5ad(precessed_adata_file_name)

scgpt
hyenadna


/home/HPCBase/tools/anaconda3/lib/python3.9/site-packages/h5py/_hl/dataset.py:767: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)
/home/HPCBase/tools/anaconda3/lib/python3.9/site-packages/h5py/_hl/dataset.py:767: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)


In [4]:
precessed_adata_file_name, sc_model_dir

('/home/share/huadjyin/home/weiyilin/project/DNALLM/datasets/raw_datasets/scrna_processed/scgpt/CD8_expression_ensembl_preprocessed.h5ad',
 '/home/share/huadjyin/home/weiyilin/project/DNALLM/scGPT/scGPT_human')

In [5]:
sc_model_cls =  pretrained_model_tokenizer_config.scrna_model_cls
sc_model_cls

omic_model.pretrained_model.scgpt._scgpt_model.ScGPTModelWrapper

In [6]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [7]:
sc_model = sc_model_cls.from_pretrained(
    pretrained_model_name_or_path=sc_model_dir,
    config_file_name="args.json",
    vocab_file_name="vocab.json", 
    model_file_name="best_model.pt",
    local_files_only=True,
    device=device,
)
sc_model.model

TransformerModel(
  (encoder): GeneEncoder(
    (embedding): Embedding(60697, 512, padding_idx=60694)
    (enc_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (value_encoder): ContinuousValueEncoder(
    (dropout): Dropout(p=0.2, inplace=False)
    (linear1): Linear(in_features=1, out_features=512, bias=True)
    (activation): ReLU()
    (linear2): Linear(in_features=512, out_features=512, bias=True)
    (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-11): 12 x FlashTransformerEncoderLayer(
        (self_attn): FlashMHA(
          (Wqkv): Linear(in_features=512, out_features=1536, bias=True)
          (inner_attn): FlashAttention()
          (out_proj): Linear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=512, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (linear2): Linear(in_features

In [8]:
next(sc_model.model.parameters()).device

device(type='cuda', index=0)

In [9]:
sc_model.vocab

GeneVocab()

In [10]:
sc_embed=sc_model.extract_sample_embedding(
    adata_or_file=preprocessed_adata,
    gene_col="gene_name",
    max_length=5000,
    cell_embedding_mode="cls",      #    could be one of "cls", "avg-pool", "w-pool"   
    batch_size = 64, 
    embedding_key = "cell_embedding",
    obs_to_save = None,
    return_new_adata=False,    
)

Embedding cells: 100%|█████████▉| 5618/5620 [02:04<00:00, 47.40it/s]/home/HPCBase/tools/anaconda3/lib/python3.9/multiprocessing/popen_fork.py:66: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x40002f034ee0>
  self.pid = os.fork()
/home/HPCBase/tools/anaconda3/lib/python3.9/multiprocessing/popen_fork.py:66: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x40002f034b20>
  self.pid = os.fork()
/home/HPCBase/tools/anaconda3/lib/python3.9/multiprocessing/popen_fork.py:66: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x40002f034ee0>
  self.pid = os.fork()
/home/HPCBase/tools/anaconda3/lib/python3.9/multiprocessing/popen_fork.py:66: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x40002f034b20>
  self.pid = os.fork()
/home/HPCBase/tools/anaconda3/lib/python3.9/multiprocessing/popen_fork.py:66: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x40002f034ee0>
  self.pid = os.fork()
/home/HPCBase/tools/anaconda3/lib/python3.9/multiprocessi

In [11]:
sc_embed

AnnData object with n_obs × n_vars = 359661 × 3495
    obs: 'library', 'sample', 'predicted_doublets', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'total_counts_ncRNA', 'pct_counts_ncRNA', 'total_counts_hb', 'pct_counts_hb', 'n_counts', 'n_genes', 'celltype_l1', 'celltype_l2', 'celltype_l3', 'celltype_l4', 'SEACell_ID', 'SEACell_l3', 'SEACell_l4', 'final_annotation'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'ensembl_id', 'gene_name', 'id_in_vocab', 'n_counts'
    uns: 'hvg', 'log1p'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap', 'bin_edges', 'cell_embedding'
    layers: 'X_binned', 'X_log1p', 'X_normed'

In [13]:
sc_embed.obsm['cell_embedding'].shape

(359661, 512)

## BY HyenaDNA

In [16]:
from datasets import load_from_disk

raw_data_config, pretrained_model_tokenizer_config = OmicRawDataConfig(), OmicPretrainedModelAndTokenizationConfig()
print(pretrained_model_tokenizer_config.scrna_model_name)
print(pretrained_model_tokenizer_config.seq_model_name)

raw_seq_dataset_path = raw_data_config.raw_seq_dataset_dir

seq_model_dir = pretrained_model_tokenizer_config.seq_model_path
seq_tokenization_dir = pretrained_model_tokenizer_config.tokenized_seq_dataset_dir


geneformer
hyenadna


In [17]:
seq_model_dir, seq_tokenization_dir

('/home/share/huadjyin/home/weiyilin/project/DNALLM/HyenaDNA/hyenadna-small-32k-seqlen',
 '/home/share/huadjyin/home/weiyilin/project/DNALLM/datasets/tokenized_datasets/seq_dataset/hyenadna')

In [18]:
seq_model_cls = pretrained_model_tokenizer_config.seq_model_cls
seq_model_cls

omic_model.pretrained_model.hyenadna._hyenadna_model.HyenaDNAPreTrainedModel

In [19]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [23]:
import os
(
os.path.dirname('/home/share/huadjyin/home/weiyilin/project/DNALLM/HyenaDNA/hyenadna-small-32k-seqlen'),
('/home/share/huadjyin/home/weiyilin/project/DNALLM/HyenaDNA/hyenadna-small-32k-seqlen').split('/')[-1]
)

('/home/share/huadjyin/home/weiyilin/project/DNALLM/HyenaDNA',
 'hyenadna-small-32k-seqlen')

In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sc_model = seq_model_cls.from_pretrained(
    seq_model_dir,
    download=False,
    config=None,
    device=device,
    use_head=False,
    n_classes=2,
    local_files_only=True,
)
sc_model.model


# DNABERT2

In [13]:
from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM
import torch

device = "cuda:1" if torch.cuda.is_available() else "cpu"

model_path = "/home/share/huadjyin/home/weiyilin/project/DNALLM/DNABERT2/DNABERT-2-117M"

tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True, trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, local_files_only=True, trust_remote_code=True)

model = model.to(device)


/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertModel were not initialized from the model checkpoint at /home/share/huadjyin/home/weiyilin/project/DNALLM/DNABERT2/DNABERT-2-117M and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(4096, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertUnpadAttention(
          (self): BertUnpadSelfAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (Wqkv): Linear(in_features=768, out_features=2304, bias=True)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (mlp): BertGatedLinearUnitMLP(
          (gated_layers): Linear(in_features=768, out_features=6144, bias=False)
          (act): GELU(approximate='none')
          (wo): L

In [22]:
dna = "ACGTAGCATCGGATCTATCTATCGACACTTGGTTATCGATCTACGAGCATCTCGTTAGC"
inputs = tokenizer(dna, return_tensors = 'pt')["input_ids"]

In [23]:
inputs

tensor([[   1,    5,  194,   32,  757, 1239, 2092,  294,   24,  359,   88,   93,
           32,   75,   77,   19,    2]])

In [24]:
inputs = inputs.to(device)

In [25]:
hidden_states = model(inputs)[0]

# not work

CompilationError: at 114:24:        else:
            if EVEN_HEADDIM:
                k = tl.load(k_ptrs + start_n * stride_kn,
                            mask=(start_n + offs_n)[:, None] < seqlen_k,
                            other=0.0)
            else:
                k = tl.load(k_ptrs + start_n * stride_kn,
                            mask=((start_n + offs_n)[:, None] < seqlen_k) &
                            (offs_d[None, :] < headdim),
                            other=0.0)
        qk = tl.zeros([BLOCK_M, BLOCK_N], dtype=tl.float32)
        qk += tl.dot(q, k, trans_b=True)
                        ^
TypeError("dot() got an unexpected keyword argument 'trans_b'")

In [12]:
import triton
triton.__version__

'2.1.0'

# NT

In [27]:
from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM
import torch

device = "cuda:1" if torch.cuda.is_available() else "cpu"

# load model from local disk
model_path = "/home/share/huadjyin/home/weiyilin/project/DNALLM/NT/nucleotide-transformer-2.5b-1000g"
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True, trust_remote_code=True)
# model = AutoModel.from_pretrained(LOCAL_MODEL_PATH_NT, local_files_only=True)
model = AutoModelForMaskedLM.from_pretrained(model_path, local_files_only=True, trust_remote_code=True)

model = model.to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [28]:
dna = "ACGTAGCATCGGATCTATCTATCGACACTTGGTTATCGATCTACGAGCATCTCGTTAGC"
inputs = tokenizer(dna, return_tensors = 'pt')["input_ids"]
inputs = inputs.to(device)

In [30]:
inputs

tensor([[   3,  727, 2163,  405, 2335,  553, 3925, 2845,  722,  415, 4101, 4101,
         4100, 4103, 4102]], device='cuda:1')

In [31]:
x = model(inputs)

In [35]:
model

EsmForMaskedLM(
  (esm): EsmModel(
    (embeddings): EsmEmbeddings(
      (word_embeddings): Embedding(4105, 2560, padding_idx=1)
      (dropout): Dropout(p=0.0, inplace=False)
      (position_embeddings): Embedding(1002, 2560, padding_idx=1)
    )
    (encoder): EsmEncoder(
      (layer): ModuleList(
        (0-31): 32 x EsmLayer(
          (attention): EsmAttention(
            (self): EsmSelfAttention(
              (query): Linear(in_features=2560, out_features=2560, bias=True)
              (key): Linear(in_features=2560, out_features=2560, bias=True)
              (value): Linear(in_features=2560, out_features=2560, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): EsmSelfOutput(
              (dense): Linear(in_features=2560, out_features=2560, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (LayerNorm): LayerNorm((2560,), eps=1e-12, elementwise_affine=True)
          )
          (int

In [34]:
x.logits.size()

torch.Size([1, 15, 4105])

In [36]:
x

MaskedLMOutput(loss=None, logits=tensor([[[-130.6536, -130.6588, -130.6781,  ..., -112.3619, -114.4805,
          -113.8212],
         [ -45.9326,  -51.7741,  -51.6475,  ...,  -50.6164,  -51.7777,
           -39.0680],
         [ -39.0530,  -36.2507,  -41.4974,  ...,  -35.0619,  -40.6025,
           -35.0656],
         ...,
         [ -36.3624,  -41.9142,  -36.4303,  ...,  -21.1460,  -22.7997,
           -25.8297],
         [ -67.5852,  -67.0025,  -68.4888,  ...,  -53.9665,  -45.8875,
           -48.5614],
         [ -38.6994,  -40.7927,  -42.1157,  ...,  -15.0632,  -21.7946,
           -23.5212]]], device='cuda:1', grad_fn=<AddBackward0>), hidden_states=None, attentions=None)

# HyenaDNA Tokenization and Embedding

In [51]:
from datasets import load_from_disk

tokenzied_ds_path = "/home/share/huadjyin/home/weiyilin/project/DNALLM/datasets/tokenized_datasets/seq_dataset/hyenadna"

tokenized_ds = load_from_disk(tokenzied_ds_path)

In [52]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['sample', 'pos', 'seq', 'peak_value', 'input_ids', 'attention_mask'],
        num_rows: 124167712
    })
    test: Dataset({
        features: ['sample', 'pos', 'seq', 'peak_value', 'input_ids', 'attention_mask'],
        num_rows: 4208429
    })
})

In [104]:
tokenized_ds['train'][0]

{'sample': 'E-B21100458292',
 'pos': 'chr1:29162-29663',
 'seq': 'ACCGGGCAGCGGCCGCGGAACACCAGCTTGGCGCAGGCTTCTCGGTCAGGAACGGTCCCGGGCCTCCCGCCCGCCTCCCTCCAGCCCCTCCGGGTCCCCTACTTCGCCCCGCCAGGCCCCCACGACCCTACTTCCCGCGGCCCCGGACGCCTCCTCACCTGCGAGCCGCCCTCCCGGAAGCTCCCGCCGCCGCTTCCGCTCTGCCGGAGCCGCTGGGTCCTAGCCCCGCCGCCCCCAGTCCGCCCGCGCCTCCGGGTCCTAACGCCGCCGCTCGCCCTCCACTGCGCCCTCCCCGAGCGCGGCTCCAGGACCCCGTCGACCCGGAGCGCTGTCCTGTCGGGCCGAGTCGCGGGCCTGGGCACGGAACTCACGCTCACTCCGAGCTCCCGACGTGCACACGGCTCCCATGCGTTGTCTTCCGAGCGTCAGGCCGCCCCTACCCGTGCTTTCTGCTCTGCAGACCCTCTTCCTAGACCTCCGTCCTTTGTCCCATCGCTGCCT',
 'peak_value': 0.0,
 'input_ids': [7,
  8,
  8,
  9,
  9,
  9,
  8,
  7,
  9,
  8,
  9,
  9,
  8,
  8,
  9,
  8,
  9,
  9,
  7,
  7,
  8,
  7,
  8,
  8,
  7,
  9,
  8,
  10,
  10,
  9,
  9,
  8,
  9,
  8,
  7,
  9,
  9,
  8,
  10,
  10,
  8,
  10,
  8,
  9,
  9,
  10,
  8,
  7,
  9,
  9,
  7,
  7,
  8,
  9,
  9,
  10,
  8,
  8,
  8,
  9,
  9,
  9,
  8,
  8,
  10,
  8,
  8,
  8,
  9,
  8,
  8,
  8,
  9,
  8,
  8,
  10,
  8,
  8

In [105]:
tokenized_ds['train'][1]

{'sample': 'E-B21100458292',
 'pos': 'chr1:94644-95145',
 'seq': 'TTCTTACAGTGGTTTCAGGCAGCATCTGAAGACAGTAAAAGCAGAAGCTCCAAGGCTTCTTACATTCTAGCCTGGAAAATCACATCACATTGCTTCCTTCATATTTTTTTGGCAAATCAGGTTGCAAGGCTTGCCCAGATTAGGGTAAAGAGGCAAAGAGGCTCCTTTTCTTTTCTTTTCTTTTTTTTTTTTTTTTTTTTTTTGAGTCAGAATCTCGCTCTGTTGCCCAGGCTGGAGTGCAGTGGTGCAATCTAGGCTCACTGCAAGCTCTGCCTCCTGGGTTCACGTCATTCTCCTGCCTCAGGCTCCCAAGTAGCTGAGACTACAGGCACCTACCACCACGCCTGGCTAATTTTTTTTTATTTTTTATTTTTTAGTAGAGACTGTGTTTCACTGTGTTAGCCAGGATGGTCTCCATCTCCTGACCTCGTGATCCTTGCAAAGGGACATGCAGACCACATTAGTGAGAATATGTGCCTGTATTTTGCAATCTGTAACATG',
 'peak_value': 0.0,
 'input_ids': [10,
  10,
  8,
  10,
  10,
  7,
  8,
  7,
  9,
  10,
  9,
  9,
  10,
  10,
  10,
  8,
  7,
  9,
  9,
  8,
  7,
  9,
  8,
  7,
  10,
  8,
  10,
  9,
  7,
  7,
  9,
  7,
  8,
  7,
  9,
  10,
  7,
  7,
  7,
  7,
  9,
  8,
  7,
  9,
  7,
  7,
  9,
  8,
  10,
  8,
  8,
  7,
  7,
  9,
  9,
  8,
  10,
  10,
  8,
  10,
  10,
  7,
  8,
  7,
  10,
  10,
  8,
  10,
  7,
  9,
  8,
  8,
  10,
  9,
  9,
  

In [103]:
tokenized_ds['train'][1]['input_ids'], len(tokenized_ds['train'][1]['input_ids'])

([10,
  10,
  8,
  10,
  10,
  7,
  8,
  7,
  9,
  10,
  9,
  9,
  10,
  10,
  10,
  8,
  7,
  9,
  9,
  8,
  7,
  9,
  8,
  7,
  10,
  8,
  10,
  9,
  7,
  7,
  9,
  7,
  8,
  7,
  9,
  10,
  7,
  7,
  7,
  7,
  9,
  8,
  7,
  9,
  7,
  7,
  9,
  8,
  10,
  8,
  8,
  7,
  7,
  9,
  9,
  8,
  10,
  10,
  8,
  10,
  10,
  7,
  8,
  7,
  10,
  10,
  8,
  10,
  7,
  9,
  8,
  8,
  10,
  9,
  9,
  7,
  7,
  7,
  7,
  10,
  8,
  7,
  8,
  7,
  10,
  8,
  7,
  8,
  7,
  10,
  10,
  9,
  8,
  10,
  10,
  8,
  8,
  10,
  10,
  8,
  7,
  10,
  7,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  9,
  9,
  8,
  7,
  7,
  7,
  10,
  8,
  7,
  9,
  9,
  10,
  10,
  9,
  8,
  7,
  7,
  9,
  9,
  8,
  10,
  10,
  9,
  8,
  8,
  8,
  7,
  9,
  7,
  10,
  10,
  7,
  9,
  9,
  9,
  10,
  7,
  7,
  7,
  9,
  7,
  9,
  9,
  8,
  7,
  7,
  7,
  9,
  7,
  9,
  9,
  8,
  10,
  8,
  8,
  10,
  10,
  10,
  10,
  8,
  10,
  10,
  10,
  10,
  8,
  10,
  10,
  10,
  10,
  8,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
 

In [56]:
raw_data_config, pretrained_model_tokenizer_config = OmicRawDataConfig(), OmicPretrainedModelAndTokenizationConfig()
print(pretrained_model_tokenizer_config.scrna_model_name)
print(pretrained_model_tokenizer_config.seq_model_name)

raw_seq_dataset_path = raw_data_config.raw_seq_dataset_dir

seq_model_dir = pretrained_model_tokenizer_config.seq_model_path
seq_tokenization_dir = pretrained_model_tokenizer_config.tokenized_seq_dataset_dir

scgpt
hyenadna


In [57]:
seq_model_dir, seq_tokenization_dir

('/home/share/huadjyin/home/weiyilin/project/DNALLM/HyenaDNA/hyenadna-small-32k-seqlen',
 '/home/share/huadjyin/home/weiyilin/project/DNALLM/datasets/tokenized_datasets/seq_dataset/hyenadna')

In [58]:
seq_model_cls = pretrained_model_tokenizer_config.seq_model_cls
seq_model_cls

omic_model.pretrained_model.hyenadna._hyenadna_model.HyenaDNAPreTrainedModel

In [59]:
import torch
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=1)

In [60]:
sc_model = seq_model_cls.from_pretrained(
    seq_model_dir,
    download=False,
    config=None,
    device=None,
    dtype=None,
    use_head=False,
)
sc_model.model

/home/share/huadjyin/home/weiyilin/project/DNALLM/omiclm/omic_model/pretrained_model/hyenadna/_hyenadna_model.py:1012: ResourceWarning: unclosed file <_io.TextIOWrapper name='/home/share/huadjyin/home/weiyilin/project/DNALLM/HyenaDNA/hyenadna-small-32k-seqlen/config.json' mode='r' encoding='UTF-8'>
  config = json.load(open(os.path.join(pretrained_model_name_or_path, 'config.json')))


HyenaDNAModel(
  (backbone): LMBackbone(
    (embeddings): GPT2Embeddings(
      (word_embeddings): Embedding(16, 256)
    )
    (layers): ModuleList(
      (0): Block(
        (mixer): HyenaOperator(
          (dropout): Dropout(p=0.0, inplace=False)
          (in_proj): Linear(in_features=256, out_features=768, bias=True)
          (out_proj): Linear(in_features=256, out_features=256, bias=True)
          (short_filter): Conv1d(768, 768, kernel_size=(3,), stride=(1,), padding=(2,), groups=768)
          (filter_fn): HyenaFilter(
            (dropout): Dropout(p=0.0, inplace=False)
            (pos_emb): PositionalEmbedding()
            (implicit_filter): Sequential(
              (0): Linear(in_features=5, out_features=64, bias=True)
              (1): Sin()
              (2): Linear(in_features=64, out_features=64, bias=True)
              (3): Sin()
              (4): Linear(in_features=64, out_features=64, bias=True)
              (5): Sin()
              (6): Linear(in_features=

In [61]:
sc_model = sc_model.to(device)

In [62]:
inputs = torch.tensor(tokenized_ds['train'][1]['input_ids']).unsqueeze(0).to(device)


In [63]:
embeding = sc_model.extract_sample_embedding(inputs)

In [64]:
embeding

tensor([[[-0.2033,  0.7969, -0.9585,  ..., -1.0996,  1.2721,  2.2758],
         [-0.2039,  0.7079, -0.8511,  ..., -1.0606,  1.3589,  2.4000],
         [ 0.2795,  0.9500, -0.5741,  ..., -1.1966,  0.9028,  1.9264],
         ...,
         [-0.9109,  0.7411, -0.7002,  ..., -1.1826,  1.6622,  2.4933],
         [-0.7485,  0.7164, -0.8480,  ..., -1.7114,  1.9717,  2.1174],
         [-0.2392,  0.5887, -0.7668,  ..., -1.3369,  1.9277,  2.2520]]],
       device='cuda:1')

In [65]:
embeding[0,99,:]

tensor([-2.2455e-01,  7.0666e-01, -8.5669e-01,  3.0253e-01,  4.3321e-01,
         5.2713e-01,  6.3142e-01, -1.8147e-01, -2.9795e-01, -5.1658e-01,
        -5.1152e+00,  1.0932e+00, -4.3119e-01,  5.4548e-01,  1.9324e+00,
         3.2238e-01, -1.3996e+00, -7.7988e-01,  3.2848e-01, -6.3507e-01,
        -5.0526e-01, -2.4330e-02, -1.7649e-01, -1.1078e+00,  1.6141e-01,
        -2.5123e-01, -7.9148e-01, -3.4256e-01,  4.4770e-02, -6.4757e-01,
        -3.4819e-01,  2.2034e-01,  7.6503e-01,  1.0582e+00,  1.3081e-01,
        -6.7668e-01,  2.6416e-03, -2.8544e-01, -5.4055e-01,  1.5608e+00,
         8.1880e-01, -9.9135e-01,  2.5708e-01, -8.9768e-01,  1.0108e+00,
        -1.2243e+00, -7.2915e-01, -9.9464e-01, -1.1928e+00, -7.3606e-01,
        -1.1682e+00, -6.4105e-01, -1.3777e+00,  6.5116e-01, -1.0453e+00,
        -5.9568e-03, -5.7955e-01, -6.4415e-01, -4.4458e-01,  2.7636e-01,
         4.4615e-01, -9.0114e-01, -3.1256e-01,  6.9016e-01, -1.0810e+00,
         1.3024e+00,  1.0131e+00,  2.4866e-01,  1.3

In [66]:
inputs = torch.tensor(tokenized_ds['train'][1]['input_ids'][:100]+[4]*100).unsqueeze(0).to(device)

In [70]:
inputs

tensor([[10, 10,  8, 10, 10,  7,  8,  7,  9, 10,  9,  9, 10, 10, 10,  8,  7,  9,
          9,  8,  7,  9,  8,  7, 10,  8, 10,  9,  7,  7,  9,  7,  8,  7,  9, 10,
          7,  7,  7,  7,  9,  8,  7,  9,  7,  7,  9,  8, 10,  8,  8,  7,  7,  9,
          9,  8, 10, 10,  8, 10, 10,  7,  8,  7, 10, 10,  8, 10,  7,  9,  8,  8,
         10,  9,  9,  7,  7,  7,  7, 10,  8,  7,  8,  7, 10,  8,  7,  8,  7, 10,
         10,  9,  8, 10, 10,  8,  8, 10, 10,  8,  4,  4,  4,  4,  4,  4,  4,  4,
          4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
          4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
          4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
          4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
          4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
          4,  4]], device='cuda:1')

In [67]:
embeding = sc_model.extract_sample_embedding(inputs)

In [68]:
embeding

tensor([[[-0.2033,  0.7969, -0.9585,  ..., -1.0996,  1.2721,  2.2758],
         [-0.2039,  0.7079, -0.8511,  ..., -1.0606,  1.3589,  2.4000],
         [ 0.2795,  0.9500, -0.5741,  ..., -1.1966,  0.9028,  1.9264],
         ...,
         [ 1.1625, -0.7446, -0.2144,  ..., -0.7632,  1.8556, -0.1802],
         [ 1.1656, -0.7387, -0.2174,  ..., -0.7597,  1.8623, -0.1804],
         [ 1.1686, -0.7329, -0.2203,  ..., -0.7563,  1.8689, -0.1806]]],
       device='cuda:1')

In [69]:
embeding[0,99,:]

tensor([-2.2455e-01,  7.0666e-01, -8.5669e-01,  3.0253e-01,  4.3321e-01,
         5.2713e-01,  6.3142e-01, -1.8147e-01, -2.9795e-01, -5.1658e-01,
        -5.1152e+00,  1.0932e+00, -4.3119e-01,  5.4548e-01,  1.9324e+00,
         3.2238e-01, -1.3996e+00, -7.7988e-01,  3.2848e-01, -6.3507e-01,
        -5.0526e-01, -2.4330e-02, -1.7649e-01, -1.1078e+00,  1.6141e-01,
        -2.5123e-01, -7.9147e-01, -3.4256e-01,  4.4770e-02, -6.4757e-01,
        -3.4819e-01,  2.2034e-01,  7.6503e-01,  1.0582e+00,  1.3081e-01,
        -6.7668e-01,  2.6421e-03, -2.8544e-01, -5.4055e-01,  1.5608e+00,
         8.1880e-01, -9.9135e-01,  2.5708e-01, -8.9768e-01,  1.0108e+00,
        -1.2243e+00, -7.2915e-01, -9.9464e-01, -1.1928e+00, -7.3606e-01,
        -1.1682e+00, -6.4105e-01, -1.3777e+00,  6.5116e-01, -1.0453e+00,
        -5.9572e-03, -5.7955e-01, -6.4415e-01, -4.4458e-01,  2.7636e-01,
         4.4615e-01, -9.0114e-01, -3.1256e-01,  6.9016e-01, -1.0810e+00,
         1.3024e+00,  1.0131e+00,  2.4866e-01,  1.3

In [37]:
inputs.device

device(type='cuda', index=1)

In [40]:
embeding, embeding.size()

(tensor([[[-0.2033,  0.7969, -0.9585,  ..., -1.0996,  1.2721,  2.2758],
          [-0.2039,  0.7079, -0.8511,  ..., -1.0606,  1.3589,  2.4000],
          [ 0.2795,  0.9500, -0.5741,  ..., -1.1966,  0.9028,  1.9264],
          ...,
          [-0.2971,  0.4829, -0.5879,  ..., -1.2084,  2.2331,  2.2380],
          [-0.5785,  0.7096, -0.5473,  ..., -1.6392,  2.0482,  2.2849],
          [-0.2246,  0.7067, -0.8567,  ..., -1.1366,  1.3522,  2.0041]]],
        device='cuda:1'),
 torch.Size([1, 100, 256]))

In [23]:
# target output size of 5
import torch.nn as nn
m = nn.AdaptiveAvgPool1d(2)
input = torch.randn(4, 3)
output = m(input.T).T
output

tensor([[ 0.8801,  0.4049,  0.1992],
        [-0.3883,  0.2222,  0.4217]])

In [97]:
output.size()

torch.Size([1, 3, 1])

In [98]:
output.transpose(2,1)

tensor([[[ 0.8004,  0.1729, -0.4327]]])

In [1]:
from datasets import Dataset, load_from_disk

x = Dataset.from_dict(
    {
        'id': [1,2,3],
        'name': ['a', 'b', 'c'],
        'val':[[1,1,1],[2,2,2],[3,3,3,3,3]]
    }
)
ds = load_from_disk('/home/share/huadjyin/home/weiyilin/project/DNALLM/datasets/embedding_datasets/scrna_dataset/scgpt/CD8_expression_5K')
x

/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Dataset({
    features: ['id', 'name', 'val'],
    num_rows: 3
})

In [2]:
x[0]

{'id': 1, 'name': 'a', 'val': [1, 1, 1]}

In [3]:
ds

Dataset({
    features: ['sample', 'cell_id', 'cell_embedding'],
    num_rows: 359661
})

In [4]:

len(ds[0]['cell_embedding'])

512

In [5]:
ds[0]

{'sample': 'E-B21712635155',
 'cell_id': 'RNA-E-B21712635155-a_2_CELL17_N1',
 'cell_embedding': [0.018159084022045135,
  0.01907915063202381,
  -0.012848003767430782,
  -0.07352319359779358,
  0.007684581447392702,
  -0.0008968673646450043,
  -0.0033184606581926346,
  -0.005018321331590414,
  -0.005796896759420633,
  0.0014917986700311303,
  -0.02228640578687191,
  -0.0626661404967308,
  -0.016939949244260788,
  0.0010634256759658456,
  -0.029526565223932266,
  -0.007700454909354448,
  -0.002326561138033867,
  0.08872604370117188,
  -0.0007825570064596832,
  -0.022632308304309845,
  0.0034180739894509315,
  0.013300016522407532,
  -0.0059492625296115875,
  0.01043566595762968,
  -0.020266009494662285,
  -0.009449183009564877,
  -0.0012235703179612756,
  -0.03194951266050339,
  0.010827003046870232,
  0.0034059081226587296,
  -0.005349723622202873,
  0.03773169964551926,
  0.03265395760536194,
  -0.01997489668428898,
  -0.03608991578221321,
  -0.03948785737156868,
  0.011064444668591022

In [4]:
filter_ds = ds.filter(lambda x: x['sample']=='E-B21106792844')

Filter:   0%|          | 0/359661 [00:00<?, ? examples/s]

In [5]:
filter_ds

Dataset({
    features: ['sample', 'cell_id', 'cell_embedding'],
    num_rows: 507
})

In [7]:
import numpy as np

np.array(filter_ds['cell_embedding']).shape

(507, 512)

In [101]:
input.mean(dim=1,keepdim=True)

tensor([[[ 0.8004,  0.1729, -0.4327]]])

In [106]:
from datasets import load_dataset

hf_ds = load_dataset("json", 
        name="seq_emb_dataset",  
        trust_remote_code=True, 
        data_files = '/home/share/huadjyin/home/weiyilin/project/DNALLM/datasets/embedding_datasets/seq_dataset/hyenadna/train/train_1.json',
        num_proc=15,
    )

Setting num_proc from 15 back to 1 for the train split to disable multiprocessing as it only contains one shard.


Generating train split: 0 examples [00:00, ? examples/s]

In [107]:
hf_ds

DatasetDict({
    train: Dataset({
        features: ['seqlen', 'peak_value', 'pos', 'sample', 'seq_embedding'],
        num_rows: 2
    })
})

In [113]:
import torch
x = torch.tensor(hf_ds['train'][0]['seq_embedding'])
x.size(), x.dtype

(torch.Size([501, 256]), torch.float32)

In [115]:
hf_ds['train']['sample']

['E-B21860921238', 'E-B21106792844']

In [116]:
124167712 // 50000

2483

In [1]:
from datasets import load_from_disk

x = load_from_disk('/home/share/huadjyin/home/weiyilin/project/DNALLM/datasets/tokenized_datasets/seq_dataset/hyenadna')

x

/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


DatasetDict({
    train: Dataset({
        features: ['sample', 'pos', 'seq', 'peak_value', 'input_ids', 'attention_mask', 'record_id'],
        num_rows: 124167712
    })
    test: Dataset({
        features: ['sample', 'pos', 'seq', 'peak_value', 'input_ids', 'attention_mask', 'record_id'],
        num_rows: 4208429
    })
})

In [12]:
x_ = x['train'].shard(num_shards=3000, index=0)

In [13]:
x_

Dataset({
    features: ['sample', 'pos', 'seq', 'peak_value', 'input_ids', 'attention_mask', 'record_id'],
    num_rows: 41390
})

In [2]:
124167712 + 4208429

128376141

In [3]:
(124167712 + 4208429)%50000, (124167712 + 4208429)//50000 

(26141, 2567)

In [15]:
111750940 / 4

27937735.0

In [19]:
import numpy as np

xx = np.int64(1)
yy = int(xx)
print(type(xx))
print(type(yy))

<class 'numpy.int64'>
<class 'int'>


In [21]:
zz = 1
print(type(zz))

<class 'int'>


In [3]:
x['train'][0]['record_id'], x['test'][-1]['record_id']

(0, 128376140)

In [10]:
from datasets import load_dataset

hf_ds = load_dataset("json", 
        name="seq_emb_dataset",  
        trust_remote_code=True, 
        data_files = '/home/share/huadjyin/home/weiyilin/project/DNALLM/datasets/embedding_datasets/seq_dataset/hyenadna/train/train_0.json.gz',
        num_proc=15,
    )

Setting num_proc from 15 back to 1 for the train split to disable multiprocessing as it only contains one shard.


Generating train split: 0 examples [00:00, ? examples/s]

In [11]:
hf_ds

DatasetDict({
    train: Dataset({
        features: ['peak_value', 'record_id', 'sample', 'seq_len', 'seq_embedding', 'pos'],
        num_rows: 4
    })
})

In [12]:
import torch

hf_ds['train']['sample']

['E-B21860921238', 'E-B21733167679', 'E-B21905045082', 'E-B21116844489']

In [13]:
hf_ds = load_dataset("json", 
        name="seq_emb_dataset",  
        trust_remote_code=True, 
        data_files = '/home/share/huadjyin/home/weiyilin/project/DNALLM/datasets/embedding_datasets/seq_dataset/hyenadna/train/train_1.json.gz',
        num_proc=15,
    )

Setting num_proc from 15 back to 1 for the train split to disable multiprocessing as it only contains one shard.


Generating train split: 0 examples [00:00, ? examples/s]

In [14]:
hf_ds

DatasetDict({
    train: Dataset({
        features: ['peak_value', 'record_id', 'sample', 'seq_len', 'seq_embedding', 'pos'],
        num_rows: 4
    })
})

In [15]:
hf_ds['train']['sample']

['E-B21106792844', 'E-B21647045540', 'E-B21421644510', 'E-B21452469585']

In [10]:
from typing import Dict, Optional, Union
from transformers import PreTrainedModel, PretrainedConfig

from omic_models import (
    PretrainedModelName,
    PRETRAINED_MODEL_NAME_CLS_MAP,
)

class OmicFormerConfig(PretrainedConfig):

    def __init__(
        self,
        seq_model_name: str = "hyenadna",
        seq_model_init_kwargs: Optional[Dict[str, Union[str, int, float]]] = None,
        seq_emb_dim: int = 256,
        scrna_model_name: str = "scgpt",
        scrna_model_init_kwargs: Optional[Dict[str, Union[str, int, float]]] = dict(
            pretrained_model_name_path=None,
            config_file_name="args.json",
            vocab_file_name="vocab.json", 
            model_file_name="best_model.pt",
            local_files_only=True,
        ),
        scrna_emb_dim: int = 512, 
        intermediate_dim: int = 768,
        n_layers_encoder: int = 4,
        n_layers_decoder: int = 4,
        n_head: int = 8,
        dropout: float = 0.1,
        **kwargs
    ):

        self.seq_model_name = seq_model_name
        self.seq_model_init_kwargs = seq_model_init_kwargs if seq_model_init_kwargs is not None else {}
        self.seq_emb_dim = seq_emb_dim
        
        self.scrna_model_name = scrna_model_name
        self.scrna_model_init_kwargs = scrna_model_init_kwargs if scrna_model_init_kwargs is not None else {}
        self.scrna_emb_dim = scrna_emb_dim

        self.intermediate_dim = intermediate_dim
        self.n_layers_encoder = n_layers_encoder
        self.n_layers_decoder = n_layers_decoder
        self.n_head = n_head
        self.dropout = dropout

        super().__init__(**kwargs)

In [11]:
omicmodel_config = OmicFormerConfig()
omicmodel_config.to_dict()

{'seq_model_name': 'hyenadna',
 'seq_model_init_kwargs': {},
 'seq_emb_dim': 256,
 'scrna_model_name': 'scgpt',
 'scrna_model_init_kwargs': {'pretrained_model_name_path': None,
  'config_file_name': 'args.json',
  'vocab_file_name': 'vocab.json',
  'model_file_name': 'best_model.pt',
  'local_files_only': True},
 'scrna_emb_dim': 512,
 'intermediate_dim': 768,
 'n_layers_encoder': 4,
 'n_layers_decoder': 4,
 'n_head': 8,
 'dropout': 0.1,
 'return_dict': True,
 'output_hidden_states': False,
 'output_attentions': False,
 'torchscript': False,
 'torch_dtype': None,
 'use_bfloat16': False,
 'tf_legacy_loss': False,
 'pruned_heads': {},
 'tie_word_embeddings': True,
 'chunk_size_feed_forward': 0,
 'is_encoder_decoder': False,
 'is_decoder': False,
 'cross_attention_hidden_size': None,
 'add_cross_attention': False,
 'tie_encoder_decoder': False,
 'max_length': 20,
 'min_length': 0,
 'do_sample': False,
 'early_stopping': False,
 'num_beams': 1,
 'num_beam_groups': 1,
 'diversity_penalty': 

In [5]:
x = PretrainedModelName('hyenadna')

In [6]:
x

<PretrainedModelName.HYENADNA: 'hyenadna'>

In [7]:
PretrainedModelName.HYENADNA

<PretrainedModelName.HYENADNA: 'hyenadna'>

In [4]:
import torch
x = torch.tensor([[1,2,3],[4,5,6]])
y = torch.tensor([[9,9,9]])

# z = torch.stack([x,y], dim=0)

In [5]:
len(y)

1

# OMIC MODEL

In [1]:
from omic_models import OmicFormer


/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
omic_former = OmicFormer(
    seq_emb_input=True,
    seq_emb_dim=110,
    scrna_emb_dim=160,
    )

In [3]:
omic_former

OmicFormer(
  (seq_embedding): OmicEmbedding(
    (pre_pooling_seq_emb): EmbeddingPooling()
  )
  (scrna_embedding): OmicEmbedding(
    (pre_pooling_seq_emb): EmbeddingPooling()
  )
  (seq_pre_encoding): Sequential(
    (0): LayerNorm((110,), eps=1e-05, elementwise_affine=True)
    (1): GatedMLP(
      (fc1): Linear(in_features=110, out_features=1536, bias=True)
      (fc2): Linear(in_features=768, out_features=768, bias=True)
    )
  )
  (scrna_pre_encoding): Sequential(
    (0): LayerNorm((160,), eps=1e-05, elementwise_affine=True)
    (1): GatedMLP(
      (fc1): Linear(in_features=160, out_features=1536, bias=True)
      (fc2): Linear(in_features=768, out_features=768, bias=True)
    )
  )
  (seq_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x EncoderBlock(
        (self_attn): MHA(
          (Wqkv): Linear(in_features=768, out_features=2304, bias=True)
          (inner_attn): SelfAttention(
            (drop): Dropout(p=0.1, inplace=False)
          )
     

In [4]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

model = omic_former 
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 244,250,781 trainable parameters


In [5]:
import torch
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

omic_former = omic_former.to(device)

In [6]:

seq_emb = torch.randn([10, 200, 110]).to(device)
scrna_emb = torch.randn([10, 300, 160]).to(device)
seq_len = torch.tensor([200]*10).to(device)

output = omic_former(
    seq_embedding=seq_emb, 
    cell_embedding=scrna_emb, 
    seq_len=seq_len)


In [7]:
output # add init

tensor([[ 0.3709],
        [ 0.0301],
        [ 0.0345],
        [-0.6893],
        [-0.0288],
        [ 0.1077],
        [ 0.1838],
        [ 0.4867],
        [-0.4004],
        [ 0.2869]], device='cuda:1', grad_fn=<AddmmBackward0>)

In [8]:
output

tensor([[-0.8976],
        [-0.8791],
        [-1.1046],
        [-0.6706],
        [-0.6881],
        [-1.1760],
        [-0.5021],
        [-1.0037],
        [-1.0162],
        [-0.6607]], device='cuda:1', grad_fn=<AddmmBackward0>)

In [7]:
output

tensor([[-0.6470],
        [-0.5095],
        [-0.3305],
        [-0.4302],
        [-0.7419],
        [-0.2869],
        [-0.5401],
        [-0.7425],
        [-0.5072],
        [-0.6115]], device='cuda:1', grad_fn=<AddmmBackward0>)

sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x400194f74c40>


# Test

In [34]:
import scanpy as sc

sc_data = sc.read_h5ad("/home/share/huadjyin/home/weiyilin/project/DNALLM/datasets/embedding_datasets/scrna_dataset/scgpt/CD8_expression_5K/CD8_expression_5K_embedding.h5ad")



In [35]:
sc_data

AnnData object with n_obs × n_vars = 359661 × 512
    obs: 'sample'

In [43]:
x=sc_data[sc_data.obs['sample']=='E-B21712635155']

In [54]:
x.obs_names.to_list()

['RNA-E-B21712635155-a_2_CELL17_N1',
 'RNA-E-B21712635155-a_2_CELL76_N3',
 'RNA-E-B21712635155-a_2_CELL114_N1',
 'RNA-E-B21712635155-a_2_CELL115_N1',
 'RNA-E-B21712635155-a_2_CELL119_N1',
 'RNA-E-B21712635155-a_2_CELL164_N1',
 'RNA-E-B21712635155-a_2_CELL201_N1',
 'RNA-E-B21712635155-a_2_CELL278_N2',
 'RNA-E-B21712635155-a_2_CELL290_N1',
 'RNA-E-B21712635155-a_2_CELL333_N1',
 'RNA-E-B21712635155-a_2_CELL351_N2',
 'RNA-E-B21712635155-a_2_CELL376_N2',
 'RNA-E-B21712635155-a_2_CELL437_N1',
 'RNA-E-B21712635155-a_2_CELL439_N1',
 'RNA-E-B21712635155-a_2_CELL457_N4',
 'RNA-E-B21712635155-a_2_CELL462_N3',
 'RNA-E-B21712635155-a_2_CELL466_N2',
 'RNA-E-B21712635155-a_2_CELL488_N1',
 'RNA-E-B21712635155-a_2_CELL511_N1',
 'RNA-E-B21712635155-a_2_CELL539_N1',
 'RNA-E-B21712635155-a_2_CELL544_N1',
 'RNA-E-B21712635155-a_2_CELL617_N2',
 'RNA-E-B21712635155-a_2_CELL647_N1',
 'RNA-E-B21712635155-a_2_CELL677_N1',
 'RNA-E-B21712635155-a_2_CELL705_N1',
 'RNA-E-B21712635155-a_2_CELL808_N2',
 'RNA-E-B21712

In [26]:
len(sc_data.obs_names[sc_data.obs['sample']=='E-B21712635155'])

978

In [47]:
'sample' in sc_data.obs.keys()

True

In [21]:
sample = sc_data.obs['sample']

In [22]:
sample[0]

/home/share/huadjyin/home/baiyong01/tmp/ipykernel_4012144/1138358554.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sample[0]


'E-B21712635155'

In [32]:
y = sc.AnnData(X=sc_data.obsm['cell_embedding'], obs=sc_data.obs[['sample']], dtype='float32')   

y

/home/share/huadjyin/home/baiyong01/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:430: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(


AnnData object with n_obs × n_vars = 359661 × 512
    obs: 'sample'

In [33]:
y.X[y.obs['sample']==,:]

SyntaxError: invalid syntax (3356514234.py, line 1)